# Analyse des algorithmes

In [1]:
# Dans cette partie nous allons tester les algorithmes avec différents jeu de données pour comparer leurs efficacités

In [2]:
# Librairies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import scipy
import scipy.stats
import time
import tensorflow as tf

# Machine learning

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

In [3]:
# Etiquetage de chaque colonne 
index = ['Fichier', 'Score', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'S7', 'S8', 'S9', 'S10', 'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18']

# Légende
#'S1':'Largeur moyenne', 
#'S2':'Écart type sur la largeur',
#'S3':'Envergure moyenne',
#'S4':'Écart type sur l’envergure',
#'S5':'Profondeur moyenne',
#'S6':'Écart-type sur la profondeur',
#'S7':'Longueur de la première et de la dernière arête',
#'S8':'Longueur moyenne de la plus grande arête',
#'S9':'Longueur de la plus grande arête de chaque tournée, divisée par la longueur de la tournée',
#'S10':'Longueur de la plus grande arête intérieure de chaque tournée (arête non connectée au dépôt), divisée par la longueur de la tournée',
#'S11':'Longueur moyenne de la première et de la dernière arête de chaque tournée',
#'S12':'Demande du premier et dernier client de chaque tournée, divisée par la charge du véhicule',
#'S13':'Demande du client le plus éloigné du dépôt, pour chaque tournée, divisée par la charge du véhicule',
#'S14':'Écart type sur la demande du client le plus éloigné du dépôt',
#'S15':'Écart type sur la longueur de chaque tournée',
#'S16':'Distance (euclidienne) moyenne entre les centre de gravité tournées',
#'S17':'Écart type sur le nombre de client de chaque tournée',
#'S18':'Degré de chaque voisinage moyen des clients'.

# Regroupement par fichier. 
keys1 = ["01", "02", "03", "04", "05", "06", "07", "08", "09","10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26", "27"]
keys2 = ["01", "02", "03", "04", "05", "06", "07", "08", "09","10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24", "25", "26"]

# Concatenation de tous les fichiers. 
files1 = glob.glob("HGS/2113/XML100_2113_*.csv")
files2 = glob.glob("HGS/2213/XML100_2213_*.csv")
files3 = glob.glob("HGS/3113/XML100_3113_*.csv")
files4 = glob.glob("HGS/3213/XML100_3213_*.csv")

data_center_random = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files1], keys = keys1)
data_center_cluster = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files2], keys = keys1)
data_coin_random = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files3], keys = keys2)
data_coin_cluster = pd.concat([pd.read_csv(f,sep=";", names = index) for f in files4], keys = keys2)

In [4]:
# Suppression de la colonne "LS7" car ne contient que des valeurs nulles.
# Suppression de la colonne "Fichier" qui ne donne aucune information sur le score. 
# Suppression de la colonne "S13" car corrélée à la colonne S12
# Suppression de la colonne "S14" car corrélée à la colonne S12 et S13
# Suppression de la colonne "S6" car corrélée à la colonne S15


data_center_random = data_center_random.drop(['Fichier','S6', 'S7', 'S13', 'S14'], axis=1)
data_center_cluster = data_center_cluster.drop(['Fichier','S6', 'S7', 'S13', 'S14'], axis=1)
data_coin_random = data_coin_random.drop(['Fichier','S6', 'S7', 'S13', 'S14'], axis=1)
data_coin_cluster = data_coin_cluster.drop(['Fichier','S6', 'S7', 'S13', 'S14'], axis=1)

In [5]:
# Sélection d'un jeu de données parmis les 26/27 présent pour chaque configuration. 
# Le jeu de donnée est sélectionné au hasard. 

# Selection de 4 jeu de données 

data_center_random_01 = data_center_random.loc["01"]
data_center_cluster_01 = data_center_cluster.loc["01"]
data_coin_random_01 = data_coin_random.loc["01"]
data_coin_cluster_01 = data_coin_cluster.loc["01"]

data_center_random_05 = data_center_random.loc["05"]
data_center_cluster_05 = data_center_cluster.loc["05"]
data_coin_random_05 = data_coin_random.loc["05"]
data_coin_cluster_05 = data_coin_cluster.loc["05"]

data_center_random_13 = data_center_random.loc["13"]
data_center_cluster_13 = data_center_cluster.loc["13"]
data_coin_random_13 = data_coin_random.loc["13"]
data_coin_cluster_13 = data_coin_cluster.loc["13"]

data_center_random_20 = data_center_random.loc["20"]
data_center_cluster_20 = data_center_cluster.loc["20"]
data_coin_random_20 = data_coin_random.loc["20"]
data_coin_cluster_20 = data_coin_cluster.loc["20"]

# Jeu issue d'un mélange

data_center_random_mix = data_center_random.sample(frac = 0.04)
data_center_cluster_mix = data_center_cluster.sample(frac = 0.04)
data_coin_random_mix = data_coin_random.sample(frac = 0.04)
data_coin_cluster_mix = data_coin_cluster.sample(frac = 0.04)

In [6]:
# Séparation du jeu de donnée 01
X_CeR_01 = data_center_random_01.iloc[:,1:]
y_CeR_01 = data_center_random_01.iloc[:,0]

X_CeC_01 = data_center_cluster_01.iloc[:,1:]
y_CeC_01 = data_center_cluster_01.iloc[:,0]

X_CoR_01 = data_coin_random_01.iloc[:,1:]
y_CoR_01 = data_coin_random_01.iloc[:,0]

X_CoC_01 = data_coin_cluster_01.iloc[:,1:]
y_CoC_01 = data_coin_cluster_01.iloc[:,0]

X_train_CeR_01, X_test_CeR_01, y_train_CeR_01, y_test_CeR_01 = train_test_split(X_CeR_01, y_CeR_01, train_size = 0.8)
X_train_CeC_01, X_test_CeC_01, y_train_CeC_01, y_test_CeC_01 = train_test_split(X_CeC_01, y_CeC_01, train_size = 0.8)
X_train_CoR_01, X_test_CoR_01, y_train_CoR_01 ,y_test_CoR_01 = train_test_split(X_CoR_01, y_CoR_01, train_size = 0.8)
X_train_CoC_01, X_test_CoC_01, y_train_CoC_01, y_test_CoC_01 = train_test_split(X_CoC_01, y_CoC_01, train_size = 0.8)

In [7]:
# Séparation du jeu de donnée 05
X_CeR_05 = data_center_random_05.iloc[:,1:]
y_CeR_05 = data_center_random_05.iloc[:,0]

X_CeC_05 = data_center_cluster_05.iloc[:,1:]
y_CeC_05 = data_center_cluster_05.iloc[:,0]

X_CoR_05 = data_coin_random_05.iloc[:,1:]
y_CoR_05 = data_coin_random_05.iloc[:,0]

X_CoC_05 = data_coin_cluster_05.iloc[:,1:]
y_CoC_05 = data_coin_cluster_05.iloc[:,0]

X_train_CeR_05, X_test_CeR_05, y_train_CeR_05, y_test_CeR_05 = train_test_split(X_CeR_05, y_CeR_05, train_size = 0.8)
X_train_CeC_05, X_test_CeC_05, y_train_CeC_05, y_test_CeC_05 = train_test_split(X_CeC_05, y_CeC_05, train_size = 0.8)
X_train_CoR_05, X_test_CoR_05, y_train_CoR_05 ,y_test_CoR_05 = train_test_split(X_CoR_05, y_CoR_05, train_size = 0.8)
X_train_CoC_05, X_test_CoC_05, y_train_CoC_05, y_test_CoC_05 = train_test_split(X_CoC_05, y_CoC_05, train_size = 0.8)

In [8]:
# Séparation du jeu de donnée 13
X_CeR_13 = data_center_random_13.iloc[:,1:]
y_CeR_13 = data_center_random_13.iloc[:,0]

X_CeC_13 = data_center_cluster_13.iloc[:,1:]
y_CeC_13 = data_center_cluster_13.iloc[:,0]

X_CoR_13 = data_coin_random_13.iloc[:,1:]
y_CoR_13 = data_coin_random_13.iloc[:,0]

X_CoC_13 = data_coin_cluster_13.iloc[:,1:]
y_CoC_13 = data_coin_cluster_13.iloc[:,0]

X_train_CeR_13, X_test_CeR_13, y_train_CeR_13, y_test_CeR_13 = train_test_split(X_CeR_13, y_CeR_13, train_size = 0.8)
X_train_CeC_13, X_test_CeC_13, y_train_CeC_13, y_test_CeC_13 = train_test_split(X_CeC_13, y_CeC_13, train_size = 0.8)
X_train_CoR_13, X_test_CoR_13, y_train_CoR_13 ,y_test_CoR_13 = train_test_split(X_CoR_13, y_CoR_13, train_size = 0.8)
X_train_CoC_13, X_test_CoC_13, y_train_CoC_13, y_test_CoC_13 = train_test_split(X_CoC_13, y_CoC_13, train_size = 0.8)

In [9]:
# Séparation du jeu de donnée 20
X_CeR_20 = data_center_random_20.iloc[:,1:]
y_CeR_20 = data_center_random_20.iloc[:,0]

X_CeC_20 = data_center_cluster_20.iloc[:,1:]
y_CeC_20 = data_center_cluster_20.iloc[:,0]

X_CoR_20 = data_coin_random_20.iloc[:,1:]
y_CoR_20 = data_coin_random_20.iloc[:,0]

X_CoC_20 = data_coin_cluster_20.iloc[:,1:]
y_CoC_20 = data_coin_cluster_20.iloc[:,0]

X_train_CeR_20, X_test_CeR_20, y_train_CeR_20, y_test_CeR_20 = train_test_split(X_CeR_20, y_CeR_20, train_size = 0.8)
X_train_CeC_20, X_test_CeC_20, y_train_CeC_20, y_test_CeC_20 = train_test_split(X_CeC_20, y_CeC_20, train_size = 0.8)
X_train_CoR_20, X_test_CoR_20, y_train_CoR_20 ,y_test_CoR_20 = train_test_split(X_CoR_20, y_CoR_20, train_size = 0.8)
X_train_CoC_20, X_test_CoC_20, y_train_CoC_20, y_test_CoC_20 = train_test_split(X_CoC_20, y_CoC_20, train_size = 0.8)

In [10]:
# Séparation du jeu de donnée mix
X_CeR_mix = data_center_random_mix.iloc[:,1:]
y_CeR_mix = data_center_random_mix.iloc[:,0]

X_CeC_mix = data_center_cluster_mix.iloc[:,1:]
y_CeC_mix = data_center_cluster_mix.iloc[:,0]

X_CoR_mix = data_coin_random_mix.iloc[:,1:]
y_CoR_mix = data_coin_random_mix.iloc[:,0]

X_CoC_mix = data_coin_cluster_mix.iloc[:,1:]
y_CoC_mix = data_coin_cluster_mix.iloc[:,0]

X_train_CeR_mix, X_test_CeR_mix, y_train_CeR_mix, y_test_CeR_mix = train_test_split(X_CeR_mix, y_CeR_mix, train_size = 0.8)
X_train_CeC_mix, X_test_CeC_mix, y_train_CeC_mix, y_test_CeC_mix = train_test_split(X_CeC_mix, y_CeC_mix, train_size = 0.8)
X_train_CoR_mix, X_test_CoR_mix, y_train_CoR_mix ,y_test_CoR_mix = train_test_split(X_CoR_mix, y_CoR_mix, train_size = 0.8)
X_train_CoC_mix, X_test_CoC_mix, y_train_CoC_mix, y_test_CoC_mix = train_test_split(X_CoC_mix, y_CoC_mix, train_size = 0.8)

In [11]:
# Mise en application sur les différents algorithmes 

## Regression linéaire 

In [12]:
from sklearn.linear_model import LinearRegression

In [57]:
model_linRegMul_CeR = LinearRegression()

# Dataset 01

model_linRegMul_CeR.fit(X_train_CeR_01, y_train_CeR_01)

precision_train_CeR_01_01 = model_linRegMul_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_linRegMul_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_linRegMul_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_linRegMul_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_linRegMul_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_linRegMul_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_linRegMul_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_linRegMul_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_linRegMul_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_linRegMul_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_linRegMul_CeR.fit(X_train_CeR_05, y_train_CeR_05)

precision_train_CeR_05_01 = model_linRegMul_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_linRegMul_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_linRegMul_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_linRegMul_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_linRegMul_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_linRegMul_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_linRegMul_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_linRegMul_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_linRegMul_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_linRegMul_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_linRegMul_CeR.fit(X_train_CeR_13, y_train_CeR_13)

precision_train_CeR_13_01 = model_linRegMul_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_linRegMul_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_linRegMul_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_linRegMul_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_linRegMul_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_linRegMul_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_linRegMul_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_linRegMul_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_linRegMul_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_linRegMul_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_linRegMul_CeR.fit(X_train_CeR_20, y_train_CeR_20)

precision_train_CeR_20_01 = model_linRegMul_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_linRegMul_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_linRegMul_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_linRegMul_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_linRegMul_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_linRegMul_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_linRegMul_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_linRegMul_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_linRegMul_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_linRegMul_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_linRegMul_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_linRegMul_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_linRegMul_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_linRegMul_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_linRegMul_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_linRegMul_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_linRegMul_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_linRegMul_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_linRegMul_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_linRegMul_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_linRegMul_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [14]:
model_linRegMul_CeC = LinearRegression()

# Dataset 01

model_linRegMul_CeC.fit(X_train_CeC_01, y_train_CeC_01)

precision_train_CeC_01_01 = model_linRegMul_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_linRegMul_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_linRegMul_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_linRegMul_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_linRegMul_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_linRegMul_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_linRegMul_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_linRegMul_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_linRegMul_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_linRegMul_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_linRegMul_CeC.fit(X_train_CeC_05, y_train_CeC_05)

precision_train_CeC_05_01 = model_linRegMul_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_linRegMul_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_linRegMul_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_linRegMul_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_linRegMul_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_linRegMul_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_linRegMul_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_linRegMul_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_linRegMul_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_linRegMul_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_linRegMul_CeC.fit(X_train_CeC_13, y_train_CeC_13)

precision_train_CeC_13_01 = model_linRegMul_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_linRegMul_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_linRegMul_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_linRegMul_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_linRegMul_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_linRegMul_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_linRegMul_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_linRegMul_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_linRegMul_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_linRegMul_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_linRegMul_CeC.fit(X_train_CeC_20, y_train_CeC_20)

precision_train_CeC_20_01 = model_linRegMul_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_linRegMul_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_linRegMul_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_linRegMul_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_linRegMul_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_linRegMul_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_linRegMul_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_linRegMul_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_linRegMul_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_linRegMul_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_linRegMul_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_linRegMul_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_linRegMul_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_linRegMul_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_linRegMul_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_linRegMul_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_linRegMul_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_linRegMul_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_linRegMul_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_linRegMul_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_linRegMul_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [15]:
model_linRegMul_CoR = LinearRegression()

# Dataset 01

model_linRegMul_CoR.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_linRegMul_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_linRegMul_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_linRegMul_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_linRegMul_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_linRegMul_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_linRegMul_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_linRegMul_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_linRegMul_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_linRegMul_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_linRegMul_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_linRegMul_CoR.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_linRegMul_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_linRegMul_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_linRegMul_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_linRegMul_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_linRegMul_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_linRegMul_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_linRegMul_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_linRegMul_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_linRegMul_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_linRegMul_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_linRegMul_CoR.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_linRegMul_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_linRegMul_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_linRegMul_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_linRegMul_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_linRegMul_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_linRegMul_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_linRegMul_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_linRegMul_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_linRegMul_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_linRegMul_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_linRegMul_CoR.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_linRegMul_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_linRegMul_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_linRegMul_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_linRegMul_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_linRegMul_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_linRegMul_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_linRegMul_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_linRegMul_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_linRegMul_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_linRegMul_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_linRegMul_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_linRegMul_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_linRegMul_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_linRegMul_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_linRegMul_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_linRegMul_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_linRegMul_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_linRegMul_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_linRegMul_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_linRegMul_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_linRegMul_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [16]:
model_linRegMul_CoC = LinearRegression()

# Dataset 01

model_linRegMul_CoC.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_linRegMul_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_linRegMul_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_linRegMul_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_linRegMul_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_linRegMul_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_linRegMul_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_linRegMul_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_linRegMul_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_linRegMul_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_linRegMul_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_linRegMul_CoC.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_linRegMul_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_linRegMul_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_linRegMul_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_linRegMul_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_linRegMul_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_linRegMul_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_linRegMul_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_linRegMul_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_linRegMul_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_linRegMul_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_linRegMul_CoC.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_linRegMul_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_linRegMul_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_linRegMul_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_linRegMul_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_linRegMul_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_linRegMul_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_linRegMul_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_linRegMul_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_linRegMul_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_linRegMul_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_linRegMul_CoC.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_linRegMul_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_linRegMul_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_linRegMul_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_linRegMul_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_linRegMul_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_linRegMul_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_linRegMul_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_linRegMul_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_linRegMul_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_linRegMul_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_linRegMul_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_linRegMul_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_linRegMul_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_linRegMul_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_linRegMul_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_linRegMul_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_linRegMul_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_linRegMul_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_linRegMul_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_linRegMul_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_linRegMul_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [17]:
data = {
        'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
       
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.845290,-37.815046,-0.655998,-111.867722,-2.052038,0.765859,-6.593169,-317.403706,-94.470227,-21.121825,0.931598,-204.740840,0.842645,-474.467868,-1.521637,0.942900,-5.578975,-61.504695,-91.591841,-23.644079
Score test_01,0.839413,-40.185651,-0.749730,-118.752229,-2.208299,0.766444,-6.686479,-319.530524,-95.118240,-21.183674,0.930635,-208.995624,0.839955,-484.819161,-1.568421,0.942724,-5.504246,-60.882645,-90.200171,-23.543389
Score train_05,-45.178004,0.790384,-96.013801,-27.902319,-1.089384,-1.262371,0.930264,-158.188246,-53.820143,-4.906528,-174.407971,0.912917,-206.359264,-139.759794,-1.869177,-28.409267,0.964220,-241.845754,-8.309801,-3.952612
Score test_05,-44.853972,0.790226,-95.374144,-27.700323,-1.075636,-1.185665,0.936231,-152.130623,-51.623046,-4.826489,-174.473618,0.913356,-206.431765,-139.782365,-1.893198,-27.431763,0.964786,-233.772782,-7.986379,-3.816799
Score train_13,-2.991847,-72.750080,0.826537,-107.187959,-2.015906,-1895.333213,-470.922763,0.779965,-3810.090506,-25.454136,-0.258665,-231.273558,0.873719,-772.929343,-6.166646,-51.442374,-25.180460,0.995376,-15.602674,-5.175652
Score test_13,-3.039098,-73.718571,0.819795,-108.550908,-2.005148,-1906.216253,-474.002401,0.778025,-3833.476981,-25.625170,-0.261438,-231.727401,0.873733,-773.976543,-6.241596,-51.345560,-25.104446,0.995409,-15.568904,-5.156749
Score train_20,-14.041382,-5.476586,-26.528429,0.915041,-0.175657,-49.874615,-10.222274,-160.115179,0.818823,-5.073118,-501.895326,-80.065557,-566.310515,0.899696,-2.741335,-607.471211,-775.941168,-233.912437,0.891675,-5.422989
Score test_20,-13.888289,-5.533785,-26.298187,0.909853,-0.143924,-49.887367,-10.232158,-160.234625,0.818852,-5.027678,-498.270442,-79.485422,-562.236918,0.899750,-2.723760,-657.437892,-840.370863,-252.963731,0.901620,-5.874214
Score train_mix,-0.371704,-0.745684,-0.973304,-0.454502,0.821934,-3.707548,-0.930250,-0.249966,-8.111043,0.840251,-1.102890,-0.011365,-1.406802,-1.893819,0.949690,0.632791,0.455375,0.190994,0.574008,0.943226
Score test_mix,-0.388340,-0.715050,-1.008327,-0.436396,0.819044,-3.812967,-0.977221,-0.248654,-8.308569,0.840310,-1.103224,-0.017092,-1.407134,-1.925918,0.948697,0.631370,0.452448,0.192151,0.572987,0.942818


# KNN Model

In [18]:
from sklearn.neighbors import KNeighborsRegressor

In [19]:
i = 10

model_KNNReg_CeR = KNeighborsRegressor(n_neighbors=i)

# Dataset 01

model_KNNReg_CeR.fit(X_train_CeR_01, y_train_CeR_01)

precision_train_CeR_01_01 = model_KNNReg_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_KNNReg_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_KNNReg_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_KNNReg_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_KNNReg_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_KNNReg_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_KNNReg_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_KNNReg_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_KNNReg_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_KNNReg_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_KNNReg_CeR.fit(X_train_CeR_05, y_train_CeR_05)

precision_train_CeR_05_01 = model_KNNReg_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_KNNReg_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_KNNReg_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_KNNReg_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_KNNReg_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_KNNReg_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_KNNReg_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_KNNReg_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_KNNReg_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_KNNReg_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_KNNReg_CeR.fit(X_train_CeR_13, y_train_CeR_13)

precision_train_CeR_13_01 = model_KNNReg_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_KNNReg_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_KNNReg_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_KNNReg_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_KNNReg_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_KNNReg_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_KNNReg_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_KNNReg_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_KNNReg_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_KNNReg_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_KNNReg_CeR.fit(X_train_CeR_20, y_train_CeR_20)

precision_train_CeR_20_01 = model_KNNReg_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_KNNReg_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_KNNReg_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_KNNReg_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_KNNReg_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_KNNReg_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_KNNReg_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_KNNReg_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_KNNReg_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_KNNReg_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_KNNReg_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_KNNReg_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_KNNReg_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_KNNReg_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_KNNReg_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_KNNReg_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_KNNReg_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_KNNReg_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_KNNReg_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_KNNReg_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_KNNReg_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [20]:
i = 10

model_KNNReg_CeC = KNeighborsRegressor(n_neighbors=i)

# Dataset 01

model_KNNReg_CeC.fit(X_train_CeC_01, y_train_CeC_01)

precision_train_CeC_01_01 = model_KNNReg_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_KNNReg_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_KNNReg_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_KNNReg_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_KNNReg_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_KNNReg_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_KNNReg_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_KNNReg_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_KNNReg_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_KNNReg_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_KNNReg_CeC.fit(X_train_CeC_05, y_train_CeC_05)

precision_train_CeC_05_01 = model_KNNReg_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_KNNReg_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_KNNReg_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_KNNReg_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_KNNReg_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_KNNReg_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_KNNReg_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_KNNReg_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_KNNReg_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_KNNReg_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_KNNReg_CeC.fit(X_train_CeC_13, y_train_CeC_13)

precision_train_CeC_13_01 = model_KNNReg_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_KNNReg_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_KNNReg_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_KNNReg_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_KNNReg_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_KNNReg_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_KNNReg_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_KNNReg_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_KNNReg_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_KNNReg_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_KNNReg_CeC.fit(X_train_CeC_20, y_train_CeC_20)

precision_train_CeC_20_01 = model_KNNReg_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_KNNReg_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_KNNReg_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_KNNReg_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_KNNReg_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_KNNReg_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_KNNReg_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_KNNReg_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_KNNReg_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_KNNReg_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_KNNReg_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_KNNReg_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_KNNReg_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_KNNReg_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_KNNReg_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_KNNReg_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_KNNReg_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_KNNReg_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_KNNReg_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_KNNReg_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_KNNReg_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [21]:
i = 10

model_KNNReg_CoR = KNeighborsRegressor(n_neighbors=i)

# Dataset 01

model_KNNReg_CoR.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_KNNReg_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_KNNReg_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_KNNReg_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_KNNReg_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_KNNReg_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_KNNReg_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_KNNReg_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_KNNReg_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_KNNReg_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_KNNReg_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_KNNReg_CoR.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_KNNReg_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_KNNReg_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_KNNReg_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_KNNReg_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_KNNReg_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_KNNReg_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_KNNReg_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_KNNReg_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_KNNReg_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_KNNReg_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_KNNReg_CoR.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_KNNReg_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_KNNReg_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_KNNReg_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_KNNReg_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_KNNReg_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_KNNReg_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_KNNReg_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_KNNReg_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_KNNReg_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_KNNReg_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_KNNReg_CoR.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_KNNReg_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_KNNReg_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_KNNReg_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_KNNReg_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_KNNReg_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_KNNReg_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_KNNReg_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_KNNReg_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_KNNReg_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_KNNReg_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_KNNReg_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_KNNReg_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_KNNReg_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_KNNReg_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_KNNReg_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_KNNReg_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_KNNReg_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_KNNReg_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_KNNReg_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_KNNReg_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_KNNReg_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [22]:
i = 10

model_KNNReg_CoC = KNeighborsRegressor(n_neighbors=i)

# Dataset 01

model_KNNReg_CoC.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_KNNReg_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_KNNReg_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_KNNReg_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_KNNReg_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_KNNReg_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_KNNReg_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_KNNReg_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_KNNReg_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_KNNReg_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_KNNReg_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_KNNReg_CoC.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_KNNReg_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_KNNReg_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_KNNReg_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_KNNReg_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_KNNReg_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_KNNReg_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_KNNReg_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_KNNReg_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_KNNReg_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_KNNReg_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_KNNReg_CoC.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_KNNReg_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_KNNReg_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_KNNReg_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_KNNReg_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_KNNReg_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_KNNReg_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_KNNReg_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_KNNReg_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_KNNReg_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_KNNReg_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_KNNReg_CoC.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_KNNReg_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_KNNReg_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_KNNReg_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_KNNReg_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_KNNReg_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_KNNReg_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_KNNReg_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_KNNReg_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_KNNReg_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_KNNReg_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_KNNReg_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_KNNReg_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_KNNReg_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_KNNReg_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_KNNReg_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_KNNReg_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_KNNReg_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_KNNReg_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_KNNReg_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_KNNReg_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_KNNReg_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [23]:
# Bilan : 

data = {'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
        
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
        
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])


,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.735033,-61.471214,-3.164690,-57.563378,-0.175409,0.911631,-0.607819,-219.927061,-20.941746,0.470640,0.903905,-173.521500,-2.159564,-548.215709,-0.415570,0.956646,-659.203184,-1698.683735,-1805.367353,0.908453
Score test_01,0.664014,-65.685718,-3.275099,-61.337144,-0.215598,0.889938,-0.614833,-221.357495,-21.098399,0.446585,0.879193,-177.259311,-2.201237,-559.828150,-0.466317,0.945930,-651.748755,-1679.640612,-1785.076084,0.908014
Score train_05,-21.820311,0.787239,-49.287285,-0.104996,0.150665,0.292831,0.961017,-122.483948,-18.244695,0.225134,-135.603092,0.855225,-211.004846,-138.750641,-1.225211,-146.405520,0.964378,-180.754478,-145.532572,0.902955
Score test_05,-21.698922,0.739182,-48.987662,-0.110663,0.136810,0.308538,0.949321,-118.008792,-17.388010,0.214462,-135.667964,0.822803,-210.980156,-138.794096,-1.145978,-141.750375,0.951809,-174.706851,-140.631091,0.892499
Score train_13,-5.624031,-118.425122,0.747668,-100.141902,-1.386308,-991.481612,-1072.136627,0.802193,-1498.121746,0.616091,-46.937044,-374.339380,0.831611,-1054.311499,0.683230,-851.905239,-119.185765,0.988442,-3.331327,0.975067
Score test_13,-5.655259,-119.748349,0.687329,-101.304342,-1.345285,-996.920030,-1077.723724,0.754041,-1506.602288,0.611861,-47.072838,-375.043271,0.791912,-1056.348565,0.698567,-850.636890,-119.043535,0.985800,-3.347605,0.975559
Score train_20,-11.497810,-0.697083,-25.667243,0.813427,-0.616822,-0.531415,-2.933310,-145.217415,0.915754,0.747471,-550.192854,-113.091688,-631.193850,0.869336,-1.418559,-3867.713769,-213.184254,-151.878639,0.908732,0.828135
Score test_20,-11.370827,-0.700850,-25.312901,0.765289,-0.515041,-0.529863,-2.917637,-145.345249,0.903527,0.753462,-546.398726,-112.211248,-626.700858,0.841859,-1.285576,-4188.022333,-230.727399,-164.671259,0.883351,0.833156
Score train_mix,-0.064718,-0.952379,-0.494392,-0.655205,0.902369,-1.407123,-1.073623,-0.259153,-2.280403,0.997636,-1.139989,-0.068668,-1.714940,-2.266542,0.976736,-0.652673,-0.400101,-2.538237,-2.597795,0.999227
Score test_mix,-0.078005,-0.914969,-0.526838,-0.626368,0.879211,-1.433845,-1.102213,-0.256108,-2.319177,0.997078,-1.131536,-0.074633,-1.705976,-2.307921,0.973134,-0.652311,-0.401608,-2.543803,-2.604859,0.999132


# Random Forest

In [24]:
from sklearn.ensemble import RandomForestRegressor

In [58]:
# Set Center & random

i = 6

model_RandomForest_CeR = RandomForestRegressor(max_depth=i)

# Dataset 01

model_RandomForest_CeR.fit(X_train_CeR_01, y_train_CeR_01)

importance_CeR_01 = model_RandomForest_CeR.feature_importances_

precision_train_CeR_01_01 = model_RandomForest_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_RandomForest_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_RandomForest_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_RandomForest_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_RandomForest_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_RandomForest_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_RandomForest_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_RandomForest_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_RandomForest_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_RandomForest_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_RandomForest_CeR.fit(X_train_CeR_05, y_train_CeR_05)

importance_CeR_05 = model_RandomForest_CeR.feature_importances_

precision_train_CeR_05_01 = model_RandomForest_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_RandomForest_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_RandomForest_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_RandomForest_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_RandomForest_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_RandomForest_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_RandomForest_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_RandomForest_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_RandomForest_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_RandomForest_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_RandomForest_CeR.fit(X_train_CeR_13, y_train_CeR_13)

importance_CeR_13 = model_RandomForest_CeR.feature_importances_

precision_train_CeR_13_01 = model_RandomForest_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_RandomForest_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_RandomForest_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_RandomForest_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_RandomForest_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_RandomForest_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_RandomForest_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_RandomForest_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_RandomForest_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_RandomForest_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_RandomForest_CeR.fit(X_train_CeR_20, y_train_CeR_20)

importance_CeR_20 = model_RandomForest_CeR.feature_importances_

precision_train_CeR_20_01 = model_RandomForest_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_RandomForest_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_RandomForest_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_RandomForest_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_RandomForest_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_RandomForest_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_RandomForest_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_RandomForest_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_RandomForest_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_RandomForest_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_RandomForest_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

importance_CeR_mix = model_RandomForest_CeR.feature_importances_

precision_train_CeR_mix_01 = model_RandomForest_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_RandomForest_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_RandomForest_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_RandomForest_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_RandomForest_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_RandomForest_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_RandomForest_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_RandomForest_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_RandomForest_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_RandomForest_CeR.score(X_test_CeR_mix, y_test_CeR_mix)


pd.DataFrame(
    zip(X_train_CeR_01.columns, importance_CeR_01, importance_CeR_05, importance_CeR_13, importance_CeR_20, importance_CeR_mix),
    columns=["feature", "weight_CeR_01", "weight_CeR_05", "weight_CeR_13", "weight_CeR_20", "weight_CeR_mix"],
).sort_values("weight_CeR_01").reset_index(drop=True)

,feature,weight_CeR_01,weight_CeR_05,weight_CeR_13,weight_CeR_20,weight_CeR_mix
0,S9,0.000043,0.000088,0.000035,0.000027,0.475361
1,S10,0.000069,0.000208,0.000056,0.000028,0.000004
2,S8,0.001143,0.000129,0.000234,0.001008,0.000099
3,S3,0.005090,0.000513,0.013748,0.001904,0.000085
4,S2,0.007209,0.284893,0.068926,0.000373,0.000015
5,S17,0.009062,0.007321,0.005997,0.002497,0.235787
6,S12,0.009285,0.048702,0.006111,0.001915,0.163330
7,S11,0.023893,0.002604,0.002155,0.018196,0.000172
8,S18,0.025608,0.031847,0.021301,0.000393,0.000435
9,S15,0.057755,0.354580,0.087660,0.461973,0.004830


In [59]:
# Set Center & cluster

i = 6

model_RandomForest_CeC = RandomForestRegressor(max_depth=i)

# Dataset 01

model_RandomForest_CeC.fit(X_train_CeC_01, y_train_CeC_01)

importance_CeC_01 = model_RandomForest_CeC.feature_importances_

precision_train_CeC_01_01 = model_RandomForest_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_RandomForest_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_RandomForest_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_RandomForest_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_RandomForest_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_RandomForest_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_RandomForest_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_RandomForest_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_RandomForest_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_RandomForest_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_RandomForest_CeC.fit(X_train_CeC_05, y_train_CeC_05)

importance_CeC_05 = model_RandomForest_CeC.feature_importances_

precision_train_CeC_05_01 = model_RandomForest_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_RandomForest_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_RandomForest_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_RandomForest_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_RandomForest_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_RandomForest_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_RandomForest_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_RandomForest_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_RandomForest_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_RandomForest_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_RandomForest_CeC.fit(X_train_CeC_13, y_train_CeC_13)

importance_CeC_13 = model_RandomForest_CeC.feature_importances_

precision_train_CeC_13_01 = model_RandomForest_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_RandomForest_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_RandomForest_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_RandomForest_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_RandomForest_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_RandomForest_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_RandomForest_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_RandomForest_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_RandomForest_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_RandomForest_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_RandomForest_CeC.fit(X_train_CeC_20, y_train_CeC_20)

importance_CeC_20 = model_RandomForest_CeC.feature_importances_

precision_train_CeC_20_01 = model_RandomForest_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_RandomForest_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_RandomForest_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_RandomForest_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_RandomForest_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_RandomForest_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_RandomForest_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_RandomForest_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_RandomForest_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_RandomForest_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_RandomForest_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

importance_CeC_mix = model_RandomForest_CeC.feature_importances_

precision_train_CeC_mix_01 = model_RandomForest_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_RandomForest_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_RandomForest_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_RandomForest_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_RandomForest_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_RandomForest_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_RandomForest_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_RandomForest_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_RandomForest_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_RandomForest_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

pd.DataFrame(
    zip(X_train_CeC_01.columns, importance_CeC_01, importance_CeC_05, importance_CeC_13, importance_CeC_20, importance_CeC_mix),
    columns=["feature", "weight_CeC_01", "weight_CeC_05", "weight_CeC_13", "weight_CeC_20", "weight_CeC_mix"],
).sort_values("weight_CeC_01").reset_index(drop=True)

,feature,weight_CeC_01,weight_CeC_05,weight_CeC_13,weight_CeC_20,weight_CeC_mix
0,S11,0.000027,0.000392,0.006708,0.039391,0.019314
1,S10,0.000055,0.000469,0.000015,0.000255,0.010811
2,S8,0.000055,0.002454,0.000055,0.002438,0.070767
3,S9,0.000061,0.000159,0.000218,0.001071,0.000026
4,S3,0.000303,0.012454,0.003394,0.079870,0.001631
5,S18,0.002882,0.007366,0.028472,0.010496,0.000010
6,S2,0.005064,0.072138,0.000269,0.020652,0.000396
7,S17,0.010097,0.376955,0.243203,0.103522,0.058481
8,S12,0.015952,0.322962,0.234178,0.102087,0.074268
9,S1,0.035765,0.015965,0.103470,0.008695,0.005945


In [27]:
# Set Coin & random

i = 6

model_RandomForest_CoR = RandomForestRegressor(max_depth=i)

# Dataset 01

model_RandomForest_CoR.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_RandomForest_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_RandomForest_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_RandomForest_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_RandomForest_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_RandomForest_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_RandomForest_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_RandomForest_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_RandomForest_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_RandomForest_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_RandomForest_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_RandomForest_CoR.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_RandomForest_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_RandomForest_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_RandomForest_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_RandomForest_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_RandomForest_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_RandomForest_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_RandomForest_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_RandomForest_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_RandomForest_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_RandomForest_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_RandomForest_CoR.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_RandomForest_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_RandomForest_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_RandomForest_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_RandomForest_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_RandomForest_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_RandomForest_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_RandomForest_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_RandomForest_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_RandomForest_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_RandomForest_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_RandomForest_CoR.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_RandomForest_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_RandomForest_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_RandomForest_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_RandomForest_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_RandomForest_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_RandomForest_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_RandomForest_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_RandomForest_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_RandomForest_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_RandomForest_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_RandomForest_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_RandomForest_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_RandomForest_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_RandomForest_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_RandomForest_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_RandomForest_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_RandomForest_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_RandomForest_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_RandomForest_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_RandomForest_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_RandomForest_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [28]:
# Set coin & cluster

i = 6

model_RandomForest_CoC = RandomForestRegressor(max_depth=i)

# Dataset 01

model_RandomForest_CoC.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_RandomForest_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_RandomForest_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_RandomForest_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_RandomForest_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_RandomForest_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_RandomForest_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_RandomForest_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_RandomForest_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_RandomForest_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_RandomForest_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_RandomForest_CoC.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_RandomForest_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_RandomForest_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_RandomForest_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_RandomForest_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_RandomForest_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_RandomForest_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_RandomForest_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_RandomForest_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_RandomForest_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_RandomForest_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_RandomForest_CoC.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_RandomForest_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_RandomForest_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_RandomForest_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_RandomForest_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_RandomForest_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_RandomForest_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_RandomForest_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_RandomForest_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_RandomForest_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_RandomForest_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_RandomForest_CoC.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_RandomForest_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_RandomForest_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_RandomForest_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_RandomForest_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_RandomForest_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_RandomForest_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_RandomForest_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_RandomForest_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_RandomForest_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_RandomForest_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_RandomForest_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_RandomForest_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_RandomForest_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_RandomForest_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_RandomForest_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_RandomForest_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_RandomForest_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_RandomForest_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_RandomForest_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_RandomForest_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_RandomForest_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [29]:
# Bilan

data = {
        'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
    
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.670182,-57.904261,-5.606688,-69.790206,-0.325877,0.834661,-1.105858,-195.764979,-48.437386,-1.573228,0.854263,-181.190629,0.081117,-558.167792,0.695988,0.914577,-1025.257040,-1917.149455,-1745.900802,0.522693
Score test_01,0.649159,-61.763671,-5.868629,-74.420974,-0.334989,0.831630,-1.120002,-197.059064,-48.746111,-1.538668,0.849576,-185.227184,0.077279,-570.054587,0.692904,0.913360,-1013.721864,-1895.595481,-1725.462732,0.528095
Score train_05,-21.026975,0.607447,-46.020968,-0.051418,-1.709084,-0.326689,0.919237,-117.808109,-32.403872,-5.106207,-126.056729,0.768130,-224.867250,-129.762880,-0.499859,-125.785771,0.947001,-164.699509,-102.781583,-1.256785
Score test_05,-20.974659,0.604486,-45.707929,-0.058903,-1.664677,-0.286579,0.920200,-113.435262,-30.971026,-4.910796,-126.044703,0.764599,-225.148253,-129.882020,-0.504061,-121.866781,0.948090,-159.178619,-99.309622,-1.232890
Score train_13,-18.405452,-123.494924,0.680331,-131.554942,-0.838735,-906.810147,-1064.394149,0.640723,-1469.619874,-4.747620,-30.704136,-332.789637,0.764564,-954.285567,0.564209,-917.625004,-39.752468,0.978244,-1.778531,0.915476
Score test_13,-18.628744,-124.838777,0.671609,-133.128764,-0.845327,-911.868709,-1069.748659,0.636406,-1477.982370,-4.774520,-30.777889,-333.362091,0.761080,-956.292507,0.564894,-916.076023,-39.706240,0.977998,-1.791146,0.920187
Score train_20,-12.948470,-0.164202,-31.885303,0.780728,-0.554637,-1.656928,-2.110850,-131.590985,0.853662,0.681132,-519.164136,-120.858900,-649.719797,0.809789,0.339674,-3952.121552,-93.273620,-183.249249,0.851372,0.134172
Score test_20,-12.782635,-0.171045,-31.433912,0.761804,-0.530701,-1.665770,-2.110077,-131.714329,0.848131,0.684086,-515.439379,-120.036405,-645.100280,0.807378,0.344331,-4279.334942,-101.353008,-198.622993,0.836365,0.136753
Score train_mix,-0.164619,-0.958239,-0.622786,-0.908173,0.935062,-1.287404,-1.377356,-0.173378,-2.793257,0.969911,-0.994199,-0.018040,-1.781712,-2.145083,0.991374,-0.693865,-0.693284,-2.735803,-2.346580,0.981472
Score test_mix,-0.175953,-0.921062,-0.655190,-0.872979,0.933828,-1.312358,-1.403902,-0.170433,-2.835990,0.970047,-0.985340,-0.024708,-1.780246,-2.183703,0.991170,-0.694807,-0.694715,-2.743627,-2.351143,0.981449


# SVM

In [30]:
from sklearn import svm

In [31]:
model_SVM_Regression = svm.SVR( kernel ='linear', gamma = 'auto')

# Dataset 01

model_SVM_Regression.fit(X_train_CeR_01, y_train_CeR_01)

precision_train_CeR_01_01 = model_SVM_Regression.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_SVM_Regression.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_SVM_Regression.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_SVM_Regression.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_SVM_Regression.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_SVM_Regression.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_SVM_Regression.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_SVM_Regression.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_SVM_Regression.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_SVM_Regression.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_SVM_Regression.fit(X_train_CeR_05, y_train_CeR_05)

precision_train_CeR_05_01 = model_SVM_Regression.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_SVM_Regression.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_SVM_Regression.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_SVM_Regression.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_SVM_Regression.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_SVM_Regression.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_SVM_Regression.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_SVM_Regression.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_SVM_Regression.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_SVM_Regression.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_SVM_Regression.fit(X_train_CeR_13, y_train_CeR_13)

precision_train_CeR_13_01 = model_SVM_Regression.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_SVM_Regression.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_SVM_Regression.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_SVM_Regression.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_SVM_Regression.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_SVM_Regression.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_SVM_Regression.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_SVM_Regression.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_SVM_Regression.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_SVM_Regression.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_SVM_Regression.fit(X_train_CeR_20, y_train_CeR_20)

precision_train_CeR_20_01 = model_SVM_Regression.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_SVM_Regression.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_SVM_Regression.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_SVM_Regression.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_SVM_Regression.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_SVM_Regression.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_SVM_Regression.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_SVM_Regression.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_SVM_Regression.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_SVM_Regression.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_SVM_Regression.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_SVM_Regression.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_SVM_Regression.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_SVM_Regression.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_SVM_Regression.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_SVM_Regression.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_SVM_Regression.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_SVM_Regression.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_SVM_Regression.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_SVM_Regression.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_SVM_Regression.score(X_test_CeR_mix, y_test_CeR_mix)

In [32]:
model_SVM_Regression = svm.SVR( kernel ='linear', gamma = 'auto')

# Dataset 01

model_SVM_Regression.fit(X_train_CeC_01, y_train_CeC_01)

precision_train_CeC_01_01 = model_SVM_Regression.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_SVM_Regression.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_SVM_Regression.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_SVM_Regression.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_SVM_Regression.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_SVM_Regression.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_SVM_Regression.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_SVM_Regression.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_SVM_Regression.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_SVM_Regression.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_SVM_Regression.fit(X_train_CeC_05, y_train_CeC_05)

precision_train_CeC_05_01 = model_SVM_Regression.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_SVM_Regression.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_SVM_Regression.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_SVM_Regression.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_SVM_Regression.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_SVM_Regression.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_SVM_Regression.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_SVM_Regression.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_SVM_Regression.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_SVM_Regression.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_SVM_Regression.fit(X_train_CeC_13, y_train_CeC_13)

precision_train_CeC_13_01 = model_SVM_Regression.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_SVM_Regression.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_SVM_Regression.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_SVM_Regression.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_SVM_Regression.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_SVM_Regression.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_SVM_Regression.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_SVM_Regression.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_SVM_Regression.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_SVM_Regression.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_SVM_Regression.fit(X_train_CeC_20, y_train_CeC_20)

precision_train_CeC_20_01 = model_SVM_Regression.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_SVM_Regression.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_SVM_Regression.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_SVM_Regression.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_SVM_Regression.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_SVM_Regression.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_SVM_Regression.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_SVM_Regression.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_SVM_Regression.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_SVM_Regression.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_SVM_Regression.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_SVM_Regression.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_SVM_Regression.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_SVM_Regression.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_SVM_Regression.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_SVM_Regression.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_SVM_Regression.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_SVM_Regression.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_SVM_Regression.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_SVM_Regression.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_SVM_Regression.score(X_test_CeC_mix, y_test_CeC_mix)

In [33]:
model_SVM_Regression = svm.SVR( kernel ='linear', gamma = 'auto')

# Dataset 01

model_SVM_Regression.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_SVM_Regression.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_SVM_Regression.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_SVM_Regression.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_SVM_Regression.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_SVM_Regression.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_SVM_Regression.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_SVM_Regression.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_SVM_Regression.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_SVM_Regression.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_SVM_Regression.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_SVM_Regression.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_SVM_Regression.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_SVM_Regression.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_SVM_Regression.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_SVM_Regression.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_SVM_Regression.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_SVM_Regression.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_SVM_Regression.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_SVM_Regression.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_SVM_Regression.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_SVM_Regression.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_SVM_Regression.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_SVM_Regression.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_SVM_Regression.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_SVM_Regression.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_SVM_Regression.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_SVM_Regression.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_SVM_Regression.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_SVM_Regression.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_SVM_Regression.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_SVM_Regression.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_SVM_Regression.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_SVM_Regression.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_SVM_Regression.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_SVM_Regression.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_SVM_Regression.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_SVM_Regression.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_SVM_Regression.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_SVM_Regression.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_SVM_Regression.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_SVM_Regression.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_SVM_Regression.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_SVM_Regression.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_SVM_Regression.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_SVM_Regression.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_SVM_Regression.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_SVM_Regression.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_SVM_Regression.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_SVM_Regression.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_SVM_Regression.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_SVM_Regression.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_SVM_Regression.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_SVM_Regression.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_SVM_Regression.score(X_test_CoR_mix, y_test_CoR_mix)

In [ ]:
model_SVM_Regression = svm.SVR( kernel ='linear', gamma = 'auto')

# Dataset 01

model_SVM_Regression.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_SVM_Regression.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_SVM_Regression.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_SVM_Regression.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_SVM_Regression.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_SVM_Regression.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_SVM_Regression.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_SVM_Regression.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_SVM_Regression.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_SVM_Regression.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_SVM_Regression.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_SVM_Regression.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_SVM_Regression.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_SVM_Regression.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_SVM_Regression.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_SVM_Regression.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_SVM_Regression.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_SVM_Regression.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_SVM_Regression.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_SVM_Regression.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_SVM_Regression.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_SVM_Regression.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_SVM_Regression.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_SVM_Regression.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_SVM_Regression.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_SVM_Regression.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_SVM_Regression.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_SVM_Regression.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_SVM_Regression.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_SVM_Regression.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_SVM_Regression.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_SVM_Regression.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_SVM_Regression.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_SVM_Regression.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_SVM_Regression.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_SVM_Regression.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_SVM_Regression.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_SVM_Regression.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_SVM_Regression.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_SVM_Regression.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_SVM_Regression.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_SVM_Regression.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_SVM_Regression.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_SVM_Regression.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_SVM_Regression.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_SVM_Regression.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_SVM_Regression.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_SVM_Regression.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_SVM_Regression.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_SVM_Regression.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_SVM_Regression.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_SVM_Regression.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_SVM_Regression.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_SVM_Regression.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_SVM_Regression.score(X_test_CoC_mix, y_test_CoC_mix)

In [35]:
# Bilan


# Tableau de bilan pour comparer les scores.

data = {'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
        
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
        
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.777888,-139.780161,-1.429776,-109.432140,-3.841082,0.711050,-18.867365,-306.885081,-79.199727,-68.616250,0.906334,-190.117837,0.851225,-435.065848,-2.596241,0.914577,-1025.257040,-1917.149455,-1745.900802,0.522693
Score test_01,0.768412,-148.428266,-1.549605,-116.194627,-4.116552,0.713856,-19.063286,-308.933366,-79.761471,-69.084925,0.904663,-194.095803,0.848739,-444.190789,-2.639804,0.913360,-1013.721864,-1895.595481,-1725.462732,0.528095
Score train_05,-19.081090,0.667663,-44.739042,-42.861282,-0.867197,0.167682,0.896383,-156.016314,-46.640928,-21.500965,-167.481135,0.873286,-199.777610,-61.216202,-1.349178,-125.785771,0.947001,-164.699509,-102.781583,-1.256785
Score test_05,-18.922340,0.659949,-44.396663,-42.482018,-0.845960,0.176268,0.896708,-150.033462,-44.694523,-20.679186,-167.524657,0.874030,-199.853530,-61.286675,-1.361874,-121.866781,0.948090,-159.178619,-99.309622,-1.232890
Score train_13,-2.385376,-184.229875,0.757295,-81.184019,-3.303784,-2677.961979,-18.376821,0.772545,-3230.885217,-2.835599,-0.513081,-212.428612,0.835462,-516.875789,-6.800652,-917.625004,-39.752468,0.978244,-1.778531,0.915476
Score test_13,-2.437805,-186.576648,0.750240,-82.288302,-3.293916,-2694.049536,-18.521101,0.770122,-3250.475862,-2.862881,-0.513570,-212.811174,0.835829,-517.779678,-6.838781,-916.076023,-39.706240,0.977998,-1.791146,0.920187
Score train_20,-7.509230,-3.309043,-18.788799,0.859336,0.052378,-112.747610,-61.216359,-159.625107,0.781905,-0.769981,-462.324979,-66.039548,-531.413769,0.885972,-1.921111,-3952.121552,-93.273620,-183.249249,0.851372,0.134172
Score test_20,-7.391120,-3.278400,-18.565393,0.855836,0.085557,-112.812346,-61.284203,-159.739714,0.785822,-0.789444,-459.017818,-65.587436,-527.638063,0.885971,-1.895876,-4279.334942,-101.353008,-198.622993,0.836365,0.136753
Score train_mix,0.128845,-1.539242,-0.228172,-0.315405,0.795741,-5.783019,0.273526,-0.234142,-6.811478,0.743411,-0.988082,0.095552,-1.280719,-1.195093,0.936630,-0.693865,-0.693284,-2.735803,-2.346580,0.981472
Score test_mix,0.121109,-1.495608,-0.250563,-0.302309,0.794247,-5.953375,0.267593,-0.232896,-6.981817,0.744464,-0.987706,0.090555,-1.280817,-1.213060,0.935647,-0.694807,-0.694715,-2.743627,-2.351143,0.981449


# Decision Tree 

In [36]:
from sklearn.tree import DecisionTreeRegressor

In [37]:
model_DecisionTree_CeR = DecisionTreeRegressor(max_depth =5,max_leaf_nodes = None, min_samples_leaf =1, min_weight_fraction_leaf =0.1,splitter = 'best')

# Dataset 01

model_DecisionTree_CeR.fit(X_train_CeR_01, y_train_CeR_01)

precision_train_CeR_01_01 = model_DecisionTree_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_DecisionTree_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_DecisionTree_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_DecisionTree_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_DecisionTree_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_DecisionTree_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_DecisionTree_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_DecisionTree_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_DecisionTree_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_DecisionTree_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_DecisionTree_CeR.fit(X_train_CeR_05, y_train_CeR_05)

precision_train_CeR_05_01 = model_DecisionTree_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_DecisionTree_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_DecisionTree_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_DecisionTree_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_DecisionTree_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_DecisionTree_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_DecisionTree_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_DecisionTree_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_DecisionTree_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_DecisionTree_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_DecisionTree_CeR.fit(X_train_CeR_13, y_train_CeR_13)

precision_train_CeR_13_01 = model_DecisionTree_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_DecisionTree_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_DecisionTree_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_DecisionTree_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_DecisionTree_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_DecisionTree_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_DecisionTree_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_DecisionTree_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_DecisionTree_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_DecisionTree_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_DecisionTree_CeR.fit(X_train_CeR_20, y_train_CeR_20)

precision_train_CeR_20_01 = model_DecisionTree_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_DecisionTree_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_DecisionTree_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_DecisionTree_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_DecisionTree_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_DecisionTree_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_DecisionTree_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_DecisionTree_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_DecisionTree_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_DecisionTree_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_DecisionTree_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_DecisionTree_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_DecisionTree_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_DecisionTree_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_DecisionTree_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_DecisionTree_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_DecisionTree_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_DecisionTree_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_DecisionTree_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_DecisionTree_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_DecisionTree_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [38]:
model_DecisionTree_CeC = DecisionTreeRegressor(max_depth =5,max_leaf_nodes = None, min_samples_leaf =1, min_weight_fraction_leaf =0.1,splitter = 'best')

# Dataset 01

model_DecisionTree_CeC.fit(X_train_CeC_01, y_train_CeC_01)

precision_train_CeC_01_01 = model_DecisionTree_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_DecisionTree_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_DecisionTree_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_DecisionTree_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_DecisionTree_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_DecisionTree_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_DecisionTree_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_DecisionTree_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_DecisionTree_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_DecisionTree_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_DecisionTree_CeC.fit(X_train_CeC_05, y_train_CeC_05)

precision_train_CeC_05_01 = model_DecisionTree_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_DecisionTree_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_DecisionTree_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_DecisionTree_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_DecisionTree_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_DecisionTree_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_DecisionTree_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_DecisionTree_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_DecisionTree_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_DecisionTree_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_DecisionTree_CeC.fit(X_train_CeC_13, y_train_CeC_13)

precision_train_CeC_13_01 = model_DecisionTree_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_DecisionTree_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_DecisionTree_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_DecisionTree_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_DecisionTree_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_DecisionTree_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_DecisionTree_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_DecisionTree_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_DecisionTree_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_DecisionTree_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_DecisionTree_CeC.fit(X_train_CeC_20, y_train_CeC_20)

precision_train_CeC_20_01 = model_DecisionTree_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_DecisionTree_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_DecisionTree_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_DecisionTree_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_DecisionTree_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_DecisionTree_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_DecisionTree_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_DecisionTree_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_DecisionTree_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_DecisionTree_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_DecisionTree_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_DecisionTree_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_DecisionTree_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_DecisionTree_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_DecisionTree_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_DecisionTree_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_DecisionTree_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_DecisionTree_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_DecisionTree_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_DecisionTree_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_DecisionTree_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [39]:
model_DecisionTree_CoR = DecisionTreeRegressor(max_depth =5,max_leaf_nodes = None, min_samples_leaf =1, min_weight_fraction_leaf =0.1,splitter = 'best')

# Dataset 01

model_DecisionTree_CoR.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_DecisionTree_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_DecisionTree_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_DecisionTree_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_DecisionTree_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_DecisionTree_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_DecisionTree_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_DecisionTree_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_DecisionTree_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_DecisionTree_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_DecisionTree_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_DecisionTree_CoR.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_DecisionTree_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_DecisionTree_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_DecisionTree_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_DecisionTree_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_DecisionTree_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_DecisionTree_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_DecisionTree_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_DecisionTree_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_DecisionTree_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_DecisionTree_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_DecisionTree_CoR.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_DecisionTree_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_DecisionTree_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_DecisionTree_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_DecisionTree_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_DecisionTree_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_DecisionTree_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_DecisionTree_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_DecisionTree_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_DecisionTree_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_DecisionTree_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_DecisionTree_CoR.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_DecisionTree_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_DecisionTree_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_DecisionTree_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_DecisionTree_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_DecisionTree_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_DecisionTree_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_DecisionTree_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_DecisionTree_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_DecisionTree_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_DecisionTree_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_DecisionTree_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_DecisionTree_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_DecisionTree_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_DecisionTree_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_DecisionTree_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_DecisionTree_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_DecisionTree_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_DecisionTree_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_DecisionTree_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_DecisionTree_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_DecisionTree_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [40]:
model_DecisionTree_CoC = DecisionTreeRegressor(max_depth =5,max_leaf_nodes = None, min_samples_leaf =1, min_weight_fraction_leaf =0.1,splitter = 'best')

# Dataset 01

model_DecisionTree_CoC.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_DecisionTree_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_DecisionTree_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_DecisionTree_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_DecisionTree_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_DecisionTree_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_DecisionTree_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_DecisionTree_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_DecisionTree_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_DecisionTree_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_DecisionTree_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_DecisionTree_CoC.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_DecisionTree_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_DecisionTree_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_DecisionTree_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_DecisionTree_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_DecisionTree_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_DecisionTree_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_DecisionTree_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_DecisionTree_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_DecisionTree_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_DecisionTree_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_DecisionTree_CoC.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_DecisionTree_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_DecisionTree_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_DecisionTree_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_DecisionTree_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_DecisionTree_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_DecisionTree_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_DecisionTree_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_DecisionTree_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_DecisionTree_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_DecisionTree_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_DecisionTree_CoC.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_DecisionTree_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_DecisionTree_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_DecisionTree_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_DecisionTree_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_DecisionTree_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_DecisionTree_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_DecisionTree_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_DecisionTree_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_DecisionTree_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_DecisionTree_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_DecisionTree_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_DecisionTree_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_DecisionTree_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_DecisionTree_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_DecisionTree_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_DecisionTree_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_DecisionTree_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_DecisionTree_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_DecisionTree_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_DecisionTree_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_DecisionTree_CoC.score(X_test_CoC_mix, y_test_CoC_mix)



In [41]:
# Bilan: 

# Tableau de bilan pour comparer les scores.

data = {
        'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
        
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
    
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])


,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.431616,-28.448335,-5.715525,-43.051598,-7.817394,0.705888,-1.249103,-235.445255,-13.735700,-30.735080,0.683123,-135.469037,-13.551286,-510.538231,-7.352227,0.808156,-639.522836,-1962.606857,-1301.312298,-137.424718
Score test_01,0.418577,-30.461004,-5.951399,-45.999446,-8.114877,0.710570,-1.259593,-236.948923,-13.825616,-31.419271,0.680853,-138.392522,-13.820831,-521.454206,-7.460211,0.809510,-632.296205,-1940.490901,-1286.647056,-135.939991
Score train_05,-14.198231,0.329790,-37.588279,-0.417202,-7.099881,0.062384,0.745103,-133.419722,-10.142624,-6.864838,-135.701841,0.581763,-257.816198,-132.519446,-4.788429,-133.253389,0.683856,-131.888088,-78.110978,-2.267481
Score test_05,-14.186217,0.323342,-37.386292,-0.410720,-7.032358,0.070822,0.741726,-128.541049,-9.652947,-6.554575,-135.688788,0.582728,-257.922932,-132.647932,-4.863539,-129.093534,0.687329,-127.444491,-75.449693,-2.220826
Score train_13,-15.039722,-71.660836,0.438676,-94.512003,-2.112832,-881.851441,-786.420719,0.442730,-1085.679282,-11.423149,-25.230151,-333.200960,0.499098,-951.812653,-4.364031,-926.613083,-148.971134,0.910069,-15.710716,-16.771724
Score test_13,-15.195876,-72.451691,0.431381,-95.530534,-2.195382,-886.783051,-791.709634,0.440096,-1091.804818,-11.431809,-25.283540,-333.923721,0.493915,-953.812998,-4.066030,-925.123991,-148.782954,0.912305,-15.702236,-16.778339
Score train_20,-9.933788,0.265847,-30.818860,0.535220,-2.141470,-3.520236,-2.394441,-158.138900,0.661732,-26.388754,-542.326386,-133.264314,-734.539031,0.650240,-40.291396,-4116.377546,-398.468221,-185.254980,0.194506,-0.331551
Score test_20,-9.793641,0.250067,-30.379653,0.526179,-2.067409,-3.523614,-2.426421,-158.266489,0.660780,-26.409622,-538.498188,-132.306360,-729.350218,0.652294,-39.521252,-4457.242045,-431.294738,-200.798662,0.216827,-0.338264
Score train_mix,-0.001133,-0.471599,-0.564714,-0.565091,0.687024,-1.212415,-0.973639,-0.324090,-1.759617,0.795843,-1.112724,0.003317,-2.329582,-1.962880,0.844978,-0.720092,-0.354103,-2.712327,-1.558940,0.875598
Score test_mix,-0.008818,-0.443633,-0.593863,-0.534979,0.686456,-1.236447,-0.995852,-0.320210,-1.789710,0.796672,-1.103031,-0.001528,-2.322797,-2.000367,0.847112,-0.720076,-0.355818,-2.719467,-1.563376,0.872147


# Gradient Boosting

In [42]:
from sklearn.ensemble import GradientBoostingRegressor

In [43]:
model_GradientBoost_CeR = GradientBoostingRegressor()

# Dataset 01

model_GradientBoost_CeR.fit(X_train_CeR_01, y_train_CeR_01)

precision_train_CeR_01_01 = model_GradientBoost_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_01_01 = model_GradientBoost_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_01_05 = model_GradientBoost_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_01_05 = model_GradientBoost_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_01_13 = model_GradientBoost_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_01_13 = model_GradientBoost_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_01_20 = model_GradientBoost_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_01_20 = model_GradientBoost_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_01_mix = model_GradientBoost_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_01_mix = model_GradientBoost_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 05

model_GradientBoost_CeR.fit(X_train_CeR_05, y_train_CeR_05)

precision_train_CeR_05_01 = model_GradientBoost_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_05_01 = model_GradientBoost_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_05_05 = model_GradientBoost_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_05_05 = model_GradientBoost_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_05_13 = model_GradientBoost_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_05_13 = model_GradientBoost_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_05_20 = model_GradientBoost_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_05_20 = model_GradientBoost_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_05_mix = model_GradientBoost_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_05_mix = model_GradientBoost_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 13

model_GradientBoost_CeR.fit(X_train_CeR_13, y_train_CeR_13)

precision_train_CeR_13_01 = model_GradientBoost_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_13_01 = model_GradientBoost_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_13_05 = model_GradientBoost_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_13_05 = model_GradientBoost_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_13_13 = model_GradientBoost_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_13_13 = model_GradientBoost_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_13_20 = model_GradientBoost_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_13_20 = model_GradientBoost_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_13_mix = model_GradientBoost_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_13_mix = model_GradientBoost_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset 20

model_GradientBoost_CeR.fit(X_train_CeR_20, y_train_CeR_20)

precision_train_CeR_20_01 = model_GradientBoost_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_20_01 = model_GradientBoost_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_20_05 = model_GradientBoost_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_20_05 = model_GradientBoost_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_20_13 = model_GradientBoost_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_20_13 = model_GradientBoost_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_20_20 = model_GradientBoost_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_20_20 = model_GradientBoost_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_20_mix = model_GradientBoost_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_20_mix = model_GradientBoost_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

# Dataset mix

model_GradientBoost_CeR.fit(X_train_CeR_mix, y_train_CeR_mix)

precision_train_CeR_mix_01 = model_GradientBoost_CeR.score(X_train_CeR_01, y_train_CeR_01)
precision_test_CeR_mix_01 = model_GradientBoost_CeR.score(X_test_CeR_01, y_test_CeR_01)

precision_train_CeR_mix_05 = model_GradientBoost_CeR.score(X_train_CeR_05, y_train_CeR_05)
precision_test_CeR_mix_05 = model_GradientBoost_CeR.score(X_test_CeR_05, y_test_CeR_05)

precision_train_CeR_mix_13 = model_GradientBoost_CeR.score(X_train_CeR_13, y_train_CeR_13)
precision_test_CeR_mix_13 = model_GradientBoost_CeR.score(X_test_CeR_13, y_test_CeR_13)

precision_train_CeR_mix_20 = model_GradientBoost_CeR.score(X_train_CeR_20, y_train_CeR_20)
precision_test_CeR_mix_20 = model_GradientBoost_CeR.score(X_test_CeR_20, y_test_CeR_20)

precision_train_CeR_mix_mix = model_GradientBoost_CeR.score(X_train_CeR_mix, y_train_CeR_mix)
precision_test_CeR_mix_mix = model_GradientBoost_CeR.score(X_test_CeR_mix, y_test_CeR_mix)

In [44]:
model_GradientBoost_CeC = GradientBoostingRegressor()

# Dataset 01

model_GradientBoost_CeC.fit(X_train_CeC_01, y_train_CeC_01)

precision_train_CeC_01_01 = model_GradientBoost_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_01_01 = model_GradientBoost_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_01_05 = model_GradientBoost_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_01_05 = model_GradientBoost_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_01_13 = model_GradientBoost_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_01_13 = model_GradientBoost_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_01_20 = model_GradientBoost_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_01_20 = model_GradientBoost_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_01_mix = model_GradientBoost_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_01_mix = model_GradientBoost_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 05

model_GradientBoost_CeC.fit(X_train_CeC_05, y_train_CeC_05)

precision_train_CeC_05_01 = model_GradientBoost_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_05_01 = model_GradientBoost_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_05_05 = model_GradientBoost_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_05_05 = model_GradientBoost_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_05_13 = model_GradientBoost_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_05_13 = model_GradientBoost_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_05_20 = model_GradientBoost_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_05_20 = model_GradientBoost_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_05_mix = model_GradientBoost_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_05_mix = model_GradientBoost_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 13

model_GradientBoost_CeC.fit(X_train_CeC_13, y_train_CeC_13)

precision_train_CeC_13_01 = model_GradientBoost_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_13_01 = model_GradientBoost_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_13_05 = model_GradientBoost_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_13_05 = model_GradientBoost_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_13_13 = model_GradientBoost_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_13_13 = model_GradientBoost_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_13_20 = model_GradientBoost_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_13_20 = model_GradientBoost_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_13_mix = model_GradientBoost_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_13_mix = model_GradientBoost_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset 20

model_GradientBoost_CeC.fit(X_train_CeC_20, y_train_CeC_20)

precision_train_CeC_20_01 = model_GradientBoost_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_20_01 = model_GradientBoost_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_20_05 = model_GradientBoost_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_20_05 = model_GradientBoost_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_20_13 = model_GradientBoost_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_20_13 = model_GradientBoost_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_20_20 = model_GradientBoost_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_20_20 = model_GradientBoost_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_20_mix = model_GradientBoost_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_20_mix = model_GradientBoost_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

# Dataset mix

model_GradientBoost_CeC.fit(X_train_CeC_mix, y_train_CeC_mix)

precision_train_CeC_mix_01 = model_GradientBoost_CeC.score(X_train_CeC_01, y_train_CeC_01)
precision_test_CeC_mix_01 = model_GradientBoost_CeC.score(X_test_CeC_01, y_test_CeC_01)

precision_train_CeC_mix_05 = model_GradientBoost_CeC.score(X_train_CeC_05, y_train_CeC_05)
precision_test_CeC_mix_05 = model_GradientBoost_CeC.score(X_test_CeC_05, y_test_CeC_05)

precision_train_CeC_mix_13 = model_GradientBoost_CeC.score(X_train_CeC_13, y_train_CeC_13)
precision_test_CeC_mix_13 = model_GradientBoost_CeC.score(X_test_CeC_13, y_test_CeC_13)

precision_train_CeC_mix_20 = model_GradientBoost_CeC.score(X_train_CeC_20, y_train_CeC_20)
precision_test_CeC_mix_20 = model_GradientBoost_CeC.score(X_test_CeC_20, y_test_CeC_20)

precision_train_CeC_mix_mix = model_GradientBoost_CeC.score(X_train_CeC_mix, y_train_CeC_mix)
precision_test_CeC_mix_mix = model_GradientBoost_CeC.score(X_test_CeC_mix, y_test_CeC_mix)

In [45]:
model_GradientBoost_CoR = GradientBoostingRegressor()

# Dataset 01

model_GradientBoost_CoR.fit(X_train_CoR_01, y_train_CoR_01)

precision_train_CoR_01_01 = model_GradientBoost_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_01_01 = model_GradientBoost_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_01_05 = model_GradientBoost_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_01_05 = model_GradientBoost_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_01_13 = model_GradientBoost_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_01_13 = model_GradientBoost_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_01_20 = model_GradientBoost_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_01_20 = model_GradientBoost_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_01_mix = model_GradientBoost_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_01_mix = model_GradientBoost_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 05

model_GradientBoost_CoR.fit(X_train_CoR_05, y_train_CoR_05)

precision_train_CoR_05_01 = model_GradientBoost_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_05_01 = model_GradientBoost_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_05_05 = model_GradientBoost_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_05_05 = model_GradientBoost_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_05_13 = model_GradientBoost_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_05_13 = model_GradientBoost_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_05_20 = model_GradientBoost_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_05_20 = model_GradientBoost_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_05_mix = model_GradientBoost_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_05_mix = model_GradientBoost_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 13

model_GradientBoost_CoR.fit(X_train_CoR_13, y_train_CoR_13)

precision_train_CoR_13_01 = model_GradientBoost_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_13_01 = model_GradientBoost_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_13_05 = model_GradientBoost_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_13_05 = model_GradientBoost_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_13_13 = model_GradientBoost_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_13_13 = model_GradientBoost_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_13_20 = model_GradientBoost_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_13_20 = model_GradientBoost_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_13_mix = model_GradientBoost_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_13_mix = model_GradientBoost_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset 20

model_GradientBoost_CoR.fit(X_train_CoR_20, y_train_CoR_20)

precision_train_CoR_20_01 = model_GradientBoost_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_20_01 = model_GradientBoost_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_20_05 = model_GradientBoost_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_20_05 = model_GradientBoost_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_20_13 = model_GradientBoost_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_20_13 = model_GradientBoost_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_20_20 = model_GradientBoost_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_20_20 = model_GradientBoost_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_20_mix = model_GradientBoost_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_20_mix = model_GradientBoost_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

# Dataset mix

model_GradientBoost_CoR.fit(X_train_CoR_mix, y_train_CoR_mix)

precision_train_CoR_mix_01 = model_GradientBoost_CoR.score(X_train_CoR_01, y_train_CoR_01)
precision_test_CoR_mix_01 = model_GradientBoost_CoR.score(X_test_CoR_01, y_test_CoR_01)

precision_train_CoR_mix_05 = model_GradientBoost_CoR.score(X_train_CoR_05, y_train_CoR_05)
precision_test_CoR_mix_05 = model_GradientBoost_CoR.score(X_test_CoR_05, y_test_CoR_05)

precision_train_CoR_mix_13 = model_GradientBoost_CoR.score(X_train_CoR_13, y_train_CoR_13)
precision_test_CoR_mix_13 = model_GradientBoost_CoR.score(X_test_CoR_13, y_test_CoR_13)

precision_train_CoR_mix_20 = model_GradientBoost_CoR.score(X_train_CoR_20, y_train_CoR_20)
precision_test_CoR_mix_20 = model_GradientBoost_CoR.score(X_test_CoR_20, y_test_CoR_20)

precision_train_CoR_mix_mix = model_GradientBoost_CoR.score(X_train_CoR_mix, y_train_CoR_mix)
precision_test_CoR_mix_mix = model_GradientBoost_CoR.score(X_test_CoR_mix, y_test_CoR_mix)

In [46]:
model_GradientBoost_CoC = GradientBoostingRegressor()

# Dataset 01

model_GradientBoost_CoC.fit(X_train_CoC_01, y_train_CoC_01)

precision_train_CoC_01_01 = model_GradientBoost_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_01_01 = model_GradientBoost_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_01_05 = model_GradientBoost_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_01_05 = model_GradientBoost_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_01_13 = model_GradientBoost_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_01_13 = model_GradientBoost_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_01_20 = model_GradientBoost_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_01_20 = model_GradientBoost_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_01_mix = model_GradientBoost_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_01_mix = model_GradientBoost_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 05

model_GradientBoost_CoC.fit(X_train_CoC_05, y_train_CoC_05)

precision_train_CoC_05_01 = model_GradientBoost_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_05_01 = model_GradientBoost_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_05_05 = model_GradientBoost_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_05_05 = model_GradientBoost_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_05_13 = model_GradientBoost_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_05_13 = model_GradientBoost_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_05_20 = model_GradientBoost_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_05_20 = model_GradientBoost_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_05_mix = model_GradientBoost_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_05_mix = model_GradientBoost_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 13

model_GradientBoost_CoC.fit(X_train_CoC_13, y_train_CoC_13)

precision_train_CoC_13_01 = model_GradientBoost_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_13_01 = model_GradientBoost_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_13_05 = model_GradientBoost_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_13_05 = model_GradientBoost_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_13_13 = model_GradientBoost_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_13_13 = model_GradientBoost_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_13_20 = model_GradientBoost_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_13_20 = model_GradientBoost_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_13_mix = model_GradientBoost_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_13_mix = model_GradientBoost_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset 20

model_GradientBoost_CoC.fit(X_train_CoC_20, y_train_CoC_20)

precision_train_CoC_20_01 = model_GradientBoost_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_20_01 = model_GradientBoost_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_20_05 = model_GradientBoost_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_20_05 = model_GradientBoost_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_20_13 = model_GradientBoost_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_20_13 = model_GradientBoost_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_20_20 = model_GradientBoost_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_20_20 = model_GradientBoost_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_20_mix = model_GradientBoost_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_20_mix = model_GradientBoost_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

# Dataset mix

model_GradientBoost_CoC.fit(X_train_CoC_mix, y_train_CoC_mix)

precision_train_CoC_mix_01 = model_GradientBoost_CoC.score(X_train_CoC_01, y_train_CoC_01)
precision_test_CoC_mix_01 = model_GradientBoost_CoC.score(X_test_CoC_01, y_test_CoC_01)

precision_train_CoC_mix_05 = model_GradientBoost_CoC.score(X_train_CoC_05, y_train_CoC_05)
precision_test_CoC_mix_05 = model_GradientBoost_CoC.score(X_test_CoC_05, y_test_CoC_05)

precision_train_CoC_mix_13 = model_GradientBoost_CoC.score(X_train_CoC_13, y_train_CoC_13)
precision_test_CoC_mix_13 = model_GradientBoost_CoC.score(X_test_CoC_13, y_test_CoC_13)

precision_train_CoC_mix_20 = model_GradientBoost_CoC.score(X_train_CoC_20, y_train_CoC_20)
precision_test_CoC_mix_20 = model_GradientBoost_CoC.score(X_test_CoC_20, y_test_CoC_20)

precision_train_CoC_mix_mix = model_GradientBoost_CoC.score(X_train_CoC_mix, y_train_CoC_mix)
precision_test_CoC_mix_mix = model_GradientBoost_CoC.score(X_test_CoC_mix, y_test_CoC_mix)

In [48]:
# Bilan 

# Tableau de bilan pour comparer les scores.

data = {
        'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
    
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.809511,-69.043457,-3.548082,-64.195423,0.358780,0.910411,-1.803164,-217.455275,-24.029167,-1.184184,0.902927,-177.209961,0.023652,-548.801186,0.657052,0.944511,-798.527709,-1923.756022,-1742.967427,0.482939
Score test_01,0.794666,-73.650716,-3.740419,-68.338567,0.337094,0.908985,-1.827714,-218.896679,-24.193034,-1.259134,0.898804,-181.090177,0.015247,-560.610436,0.649964,0.943537,-789.556190,-1902.052084,-1722.714432,0.486932
Score train_05,-15.395485,0.811534,-29.660125,0.003996,-0.485296,-3.959594,0.958339,-122.927155,-15.621619,-2.257825,-124.439411,0.866374,-217.061304,-120.953868,0.686291,-109.151211,0.973435,-172.424935,-109.782260,0.583714
Score test_05,-15.338075,0.808026,-29.486556,-0.009366,-0.484307,-3.741126,0.956161,-118.394560,-14.893641,-2.197587,-124.442337,0.863551,-217.156777,-121.078616,0.683354,-105.774542,0.973450,-166.663658,-106.079747,0.591304
Score train_13,-8.531227,-133.019171,0.784961,-107.698626,0.247324,-918.314757,-969.872078,0.776652,-1313.943985,-1.479978,-34.633490,-330.021391,0.840695,-951.682582,0.555941,-901.923246,-55.443546,0.986298,-2.083807,0.737543
Score test_13,-8.620394,-134.599781,0.775217,-108.951125,0.249313,-923.391442,-975.095153,0.770799,-1321.445524,-1.496314,-34.734012,-330.633934,0.837328,-953.566673,0.570576,-900.633015,-55.366660,0.986065,-2.087488,0.738723
Score train_20,-10.710896,-5.849646,-26.497642,0.870196,0.252013,-0.008840,-3.835477,-133.794498,0.899992,0.409934,-493.510836,-107.356049,-630.810751,0.879518,0.628663,-3837.307607,-181.546552,-151.198866,0.948103,-0.293291
Score test_20,-10.567143,-5.845100,-26.141097,0.856960,0.270509,-0.007061,-3.830261,-133.921677,0.896952,0.426694,-489.928079,-106.608169,-626.327932,0.878529,0.626168,-4155.032007,-196.998380,-163.892518,0.943736,-0.357424
Score train_mix,0.093664,-1.426245,-0.300716,-0.735002,0.962003,-1.329044,-1.398718,-0.209750,-2.316675,0.988354,-0.952594,0.004702,-1.715311,-2.083120,0.995520,-0.586005,-0.483674,-2.729551,-2.365226,0.994760
Score test_mix,0.085593,-1.378404,-0.326230,-0.699990,0.960393,-1.354037,-1.426202,-0.205517,-2.349125,0.988097,-0.944160,-0.001497,-1.712435,-2.120392,0.995396,-0.586337,-0.484360,-2.737014,-2.370112,0.994640


# Neural Network

In [49]:
input_shape = X_CeR_01.shape[1]
output_shape = 1

# we set up our input layer
inputs = Input(shape=(input_shape,))

# we add 3 hidden layers with diminishing size. This is a common practice in designing a neural network
# as the features get more and more abstracted, we need less and less neurons.

h = Dense(16, activation="relu")(inputs)
h = Dense(8, activation="relu")(h)
h = Dense(4, activation="relu")(h)

# and finally we use the ReLU activation function on the output layer
out = Dense(output_shape, activation="relu")(h)

model_NN_CeR = Sequential()
model_NN_CeR.add(Dense(units=5, input_dim=14, kernel_initializer='normal', activation='relu'))
model_NN_CeR.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
model_NN_CeR.add(Dense(1, kernel_initializer='normal'))
model_NN_CeR.compile(loss='mean_squared_error', optimizer='adam')

# Dataset 01

model_NN_CeR.fit(X_train_CeR_01, y_train_CeR_01,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeR_01=model_NN_CeR.predict(X_train_CeR_01)
Predictions_test_CeR_01=model_NN_CeR.predict(X_test_CeR_01)

Predictions_train_CeR_05=model_NN_CeR.predict(X_train_CeR_05)
Predictions_test_CeR_05=model_NN_CeR.predict(X_test_CeR_05)

Predictions_train_CeR_13=model_NN_CeR.predict(X_train_CeR_13)
Predictions_test_CeR_13=model_NN_CeR.predict(X_test_CeR_13)

Predictions_train_CeR_20=model_NN_CeR.predict(X_train_CeR_20)
Predictions_test_CeR_20=model_NN_CeR.predict(X_test_CeR_20)

Predictions_train_CeR_mix=model_NN_CeR.predict(X_train_CeR_mix)
Predictions_test_CeR_mix=model_NN_CeR.predict(X_test_CeR_mix)


precision_train_CeR_01_01 = r2_score(y_train_CeR_01,Predictions_train_CeR_01)
precision_test_CeR_01_01 = r2_score(y_test_CeR_01,Predictions_test_CeR_01)

precision_train_CeR_01_05 = r2_score(y_train_CeR_05,Predictions_train_CeR_05)
precision_test_CeR_01_05 = r2_score(y_test_CeR_05,Predictions_test_CeR_05)

precision_train_CeR_01_13 = r2_score(y_train_CeR_13,Predictions_train_CeR_13)
precision_test_CeR_01_13 = r2_score(y_test_CeR_13,Predictions_test_CeR_13)

precision_train_CeR_01_20 = r2_score(y_train_CeR_20,Predictions_train_CeR_20)
precision_test_CeR_01_20 = r2_score(y_test_CeR_20,Predictions_test_CeR_20)

precision_train_CeR_01_mix = r2_score(y_train_CeR_mix,Predictions_train_CeR_mix)
precision_test_CeR_01_mix = r2_score(y_test_CeR_mix,Predictions_test_CeR_mix)

# Dataset 05

model_NN_CeR.fit(X_train_CeR_05, y_train_CeR_05,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeR_01=model_NN_CeR.predict(X_train_CeR_01)
Predictions_test_CeR_01=model_NN_CeR.predict(X_test_CeR_01)

Predictions_train_CeR_05=model_NN_CeR.predict(X_train_CeR_05)
Predictions_test_CeR_05=model_NN_CeR.predict(X_test_CeR_05)

Predictions_train_CeR_13=model_NN_CeR.predict(X_train_CeR_13)
Predictions_test_CeR_13=model_NN_CeR.predict(X_test_CeR_13)

Predictions_train_CeR_20=model_NN_CeR.predict(X_train_CeR_20)
Predictions_test_CeR_20=model_NN_CeR.predict(X_test_CeR_20)

Predictions_train_CeR_mix=model_NN_CeR.predict(X_train_CeR_mix)
Predictions_test_CeR_mix=model_NN_CeR.predict(X_test_CeR_mix)

precision_train_CeR_05_01 = r2_score(y_train_CeR_01,Predictions_train_CeR_01)
precision_test_CeR_05_01 = r2_score(y_test_CeR_01,Predictions_test_CeR_01)

precision_train_CeR_05_05 = r2_score(y_train_CeR_05,Predictions_train_CeR_05)
precision_test_CeR_05_05 = r2_score(y_test_CeR_05,Predictions_test_CeR_05)

precision_train_CeR_05_13 = r2_score(y_train_CeR_13,Predictions_train_CeR_13)
precision_test_CeR_05_13 = r2_score(y_test_CeR_13,Predictions_test_CeR_13)

precision_train_CeR_05_20 = r2_score(y_train_CeR_20,Predictions_train_CeR_20)
precision_test_CeR_05_20 = r2_score(y_test_CeR_20,Predictions_test_CeR_20)

precision_train_CeR_05_mix = r2_score(y_train_CeR_mix,Predictions_train_CeR_mix)
precision_test_CeR_05_mix = r2_score(y_test_CeR_mix,Predictions_test_CeR_mix)

# Dataset 13

model_NN_CeR.fit(X_train_CeR_13, y_train_CeR_13,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeR_01=model_NN_CeR.predict(X_train_CeR_01)
Predictions_test_CeR_01=model_NN_CeR.predict(X_test_CeR_01)

Predictions_train_CeR_05=model_NN_CeR.predict(X_train_CeR_05)
Predictions_test_CeR_05=model_NN_CeR.predict(X_test_CeR_05)

Predictions_train_CeR_13=model_NN_CeR.predict(X_train_CeR_13)
Predictions_test_CeR_13=model_NN_CeR.predict(X_test_CeR_13)

Predictions_train_CeR_20=model_NN_CeR.predict(X_train_CeR_20)
Predictions_test_CeR_20=model_NN_CeR.predict(X_test_CeR_20)

Predictions_train_CeR_mix=model_NN_CeR.predict(X_train_CeR_mix)
Predictions_test_CeR_mix=model_NN_CeR.predict(X_test_CeR_mix)

precision_train_CeR_13_01 = r2_score(y_train_CeR_01,Predictions_train_CeR_01)
precision_test_CeR_13_01 = r2_score(y_test_CeR_01,Predictions_test_CeR_01)

precision_train_CeR_13_05 = r2_score(y_train_CeR_05,Predictions_train_CeR_05)
precision_test_CeR_13_05 = r2_score(y_test_CeR_05,Predictions_test_CeR_05)

precision_train_CeR_13_13 = r2_score(y_train_CeR_13,Predictions_train_CeR_13)
precision_test_CeR_13_13 = r2_score(y_test_CeR_13,Predictions_test_CeR_13)

precision_train_CeR_13_20 = r2_score(y_train_CeR_20,Predictions_train_CeR_20)
precision_test_CeR_13_20 = r2_score(y_test_CeR_20,Predictions_test_CeR_20)

precision_train_CeR_13_mix = r2_score(y_train_CeR_mix,Predictions_train_CeR_mix)
precision_test_CeR_13_mix = r2_score(y_test_CeR_mix,Predictions_test_CeR_mix)

# Dataset 20

model_NN_CeR.fit(X_train_CeR_20, y_train_CeR_20,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeR_01=model_NN_CeR.predict(X_train_CeR_01)
Predictions_test_CeR_01=model_NN_CeR.predict(X_test_CeR_01)

Predictions_train_CeR_05=model_NN_CeR.predict(X_train_CeR_05)
Predictions_test_CeR_05=model_NN_CeR.predict(X_test_CeR_05)

Predictions_train_CeR_13=model_NN_CeR.predict(X_train_CeR_13)
Predictions_test_CeR_13=model_NN_CeR.predict(X_test_CeR_13)

Predictions_train_CeR_20=model_NN_CeR.predict(X_train_CeR_20)
Predictions_test_CeR_20=model_NN_CeR.predict(X_test_CeR_20)

Predictions_train_CeR_mix=model_NN_CeR.predict(X_train_CeR_mix)
Predictions_test_CeR_mix=model_NN_CeR.predict(X_test_CeR_mix)

precision_train_CeR_20_01 = r2_score(y_train_CeR_01,Predictions_train_CeR_01)
precision_test_CeR_20_01 = r2_score(y_test_CeR_01,Predictions_test_CeR_01)

precision_train_CeR_20_05 = r2_score(y_train_CeR_05,Predictions_train_CeR_05)
precision_test_CeR_20_05 = r2_score(y_test_CeR_05,Predictions_test_CeR_05)

precision_train_CeR_20_13 = r2_score(y_train_CeR_13,Predictions_train_CeR_13)
precision_test_CeR_20_13 = r2_score(y_test_CeR_13,Predictions_test_CeR_13)

precision_train_CeR_20_20 = r2_score(y_train_CeR_20,Predictions_train_CeR_20)
precision_test_CeR_20_20 = r2_score(y_test_CeR_20,Predictions_test_CeR_20)

precision_train_CeR_20_mix = r2_score(y_train_CeR_mix,Predictions_train_CeR_mix)
precision_test_CeR_20_mix = r2_score(y_test_CeR_mix,Predictions_test_CeR_mix)

# Dataset mix

model_NN_CeR.fit(X_train_CeR_mix, y_train_CeR_mix,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeR_01=model_NN_CeR.predict(X_train_CeR_01)
Predictions_test_CeR_01=model_NN_CeR.predict(X_test_CeR_01)

Predictions_train_CeR_05=model_NN_CeR.predict(X_train_CeR_05)
Predictions_test_CeR_05=model_NN_CeR.predict(X_test_CeR_05)

Predictions_train_CeR_13=model_NN_CeR.predict(X_train_CeR_13)
Predictions_test_CeR_13=model_NN_CeR.predict(X_test_CeR_13)

Predictions_train_CeR_20=model_NN_CeR.predict(X_train_CeR_20)
Predictions_test_CeR_20=model_NN_CeR.predict(X_test_CeR_20)

Predictions_train_CeR_mix=model_NN_CeR.predict(X_train_CeR_mix)
Predictions_test_CeR_mix=model_NN_CeR.predict(X_test_CeR_mix)

precision_train_CeR_mix_01 = r2_score(y_train_CeR_01,Predictions_train_CeR_01)
precision_test_CeR_mix_01 = r2_score(y_test_CeR_01,Predictions_test_CeR_01)

precision_train_CeR_mix_05 = r2_score(y_train_CeR_05,Predictions_train_CeR_05)
precision_test_CeR_mix_05 = r2_score(y_test_CeR_05,Predictions_test_CeR_05)

precision_train_CeR_mix_13 = r2_score(y_train_CeR_13,Predictions_train_CeR_13)
precision_test_CeR_mix_13 = r2_score(y_test_CeR_13,Predictions_test_CeR_13)

precision_train_CeR_mix_20 = r2_score(y_train_CeR_20,Predictions_train_CeR_20)
precision_test_CeR_mix_20 = r2_score(y_test_CeR_20,Predictions_test_CeR_20)

precision_train_CeR_mix_mix = r2_score(y_train_CeR_mix,Predictions_train_CeR_mix)
precision_test_CeR_mix_mix = r2_score(y_test_CeR_mix,Predictions_test_CeR_mix)

2022-11-26 19:17:59.348957: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


635/635 [==============================] - 0s 253us/step


,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix
Score train_01,0.630612,-192.691212,0.300375,-65.706674,-3.035023
Score test_01,0.608815,-204.676989,0.254352,-69.820991,-3.252302
Score train_05,-144.563585,0.650190,-92.218183,-19.307212,-2.615155
Score test_05,-143.343578,0.643997,-91.378291,-19.130758,-2.580816
Score train_13,0.205964,-200.640783,0.731093,-63.612914,-2.352775
Score test_13,0.192428,-203.192507,0.723526,-64.437256,-2.335258
Score train_20,-15.601843,-18.057168,-12.547540,0.857383,0.175595
Score test_20,-15.411784,-17.941156,-12.380212,0.855195,0.204611
Score train_mix,-0.979499,-2.161126,-0.408077,-0.080718,0.794808
Score test_mix,-1.010008,-2.107516,-0.431533,-0.070094,0.793812


In [50]:
# Bilan : le modèle ne résiste pas 

In [51]:
input_shape = X_CeC_01.shape[1]
output_shape = 1

# we set up our input layer
inputs = Input(shape=(input_shape,))

# we add 3 hidden layers with diminishing size. This is a common practice in designing a neural network
# as the features get more and more abstracted, we need less and less neurons.

h = Dense(16, activation="relu")(inputs)
h = Dense(8, activation="relu")(h)
h = Dense(4, activation="relu")(h)

# and finally we use the ReLU activation function on the output layer
out = Dense(output_shape, activation="relu")(h)

model_NN_CeC = Sequential()
model_NN_CeC.add(Dense(units=5, input_dim=14, kernel_initializer='normal', activation='relu'))
model_NN_CeC.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
model_NN_CeC.add(Dense(1, kernel_initializer='normal'))
model_NN_CeC.compile(loss='mean_squared_error', optimizer='adam')

# Dataset 01

model_NN_CeC.fit(X_train_CeC_01, y_train_CeC_01,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeC_01=model_NN_CeC.predict(X_train_CeC_01)
Predictions_test_CeC_01=model_NN_CeC.predict(X_test_CeC_01)

Predictions_train_CeC_05=model_NN_CeC.predict(X_train_CeC_05)
Predictions_test_CeC_05=model_NN_CeC.predict(X_test_CeC_05)

Predictions_train_CeC_13=model_NN_CeC.predict(X_train_CeC_13)
Predictions_test_CeC_13=model_NN_CeC.predict(X_test_CeC_13)

Predictions_train_CeC_20=model_NN_CeC.predict(X_train_CeC_20)
Predictions_test_CeC_20=model_NN_CeC.predict(X_test_CeC_20)

Predictions_train_CeC_mix=model_NN_CeC.predict(X_train_CeC_mix)
Predictions_test_CeC_mix=model_NN_CeC.predict(X_test_CeC_mix)


precision_train_CeC_01_01 = r2_score(y_train_CeC_01,Predictions_train_CeC_01)
precision_test_CeC_01_01 = r2_score(y_test_CeC_01,Predictions_test_CeC_01)

precision_train_CeC_01_05 = r2_score(y_train_CeC_05,Predictions_train_CeC_05)
precision_test_CeC_01_05 = r2_score(y_test_CeC_05,Predictions_test_CeC_05)

precision_train_CeC_01_13 = r2_score(y_train_CeC_13,Predictions_train_CeC_13)
precision_test_CeC_01_13 = r2_score(y_test_CeC_13,Predictions_test_CeC_13)

precision_train_CeC_01_20 = r2_score(y_train_CeC_20,Predictions_train_CeC_20)
precision_test_CeC_01_20 = r2_score(y_test_CeC_20,Predictions_test_CeC_20)

precision_train_CeC_01_mix = r2_score(y_train_CeC_mix,Predictions_train_CeC_mix)
precision_test_CeC_01_mix = r2_score(y_test_CeC_mix,Predictions_test_CeC_mix)

# Dataset 05

model_NN_CeC.fit(X_train_CeC_05, y_train_CeC_05,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeC_01=model_NN_CeC.predict(X_train_CeC_01)
Predictions_test_CeC_01=model_NN_CeC.predict(X_test_CeC_01)

Predictions_train_CeC_05=model_NN_CeC.predict(X_train_CeC_05)
Predictions_test_CeC_05=model_NN_CeC.predict(X_test_CeC_05)

Predictions_train_CeC_13=model_NN_CeC.predict(X_train_CeC_13)
Predictions_test_CeC_13=model_NN_CeC.predict(X_test_CeC_13)

Predictions_train_CeC_20=model_NN_CeC.predict(X_train_CeC_20)
Predictions_test_CeC_20=model_NN_CeC.predict(X_test_CeC_20)

Predictions_train_CeC_mix=model_NN_CeC.predict(X_train_CeC_mix)
Predictions_test_CeC_mix=model_NN_CeC.predict(X_test_CeC_mix)

precision_train_CeC_05_01 = r2_score(y_train_CeC_01,Predictions_train_CeC_01)
precision_test_CeC_05_01 = r2_score(y_test_CeC_01,Predictions_test_CeC_01)

precision_train_CeC_05_05 = r2_score(y_train_CeC_05,Predictions_train_CeC_05)
precision_test_CeC_05_05 = r2_score(y_test_CeC_05,Predictions_test_CeC_05)

precision_train_CeC_05_13 = r2_score(y_train_CeC_13,Predictions_train_CeC_13)
precision_test_CeC_05_13 = r2_score(y_test_CeC_13,Predictions_test_CeC_13)

precision_train_CeC_05_20 = r2_score(y_train_CeC_20,Predictions_train_CeC_20)
precision_test_CeC_05_20 = r2_score(y_test_CeC_20,Predictions_test_CeC_20)

precision_train_CeC_05_mix = r2_score(y_train_CeC_mix,Predictions_train_CeC_mix)
precision_test_CeC_05_mix = r2_score(y_test_CeC_mix,Predictions_test_CeC_mix)

# Dataset 13

model_NN_CeC.fit(X_train_CeC_13, y_train_CeC_13,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeC_01=model_NN_CeC.predict(X_train_CeC_01)
Predictions_test_CeC_01=model_NN_CeC.predict(X_test_CeC_01)

Predictions_train_CeC_05=model_NN_CeC.predict(X_train_CeC_05)
Predictions_test_CeC_05=model_NN_CeC.predict(X_test_CeC_05)

Predictions_train_CeC_13=model_NN_CeC.predict(X_train_CeC_13)
Predictions_test_CeC_13=model_NN_CeC.predict(X_test_CeC_13)

Predictions_train_CeC_20=model_NN_CeC.predict(X_train_CeC_20)
Predictions_test_CeC_20=model_NN_CeC.predict(X_test_CeC_20)

Predictions_train_CeC_mix=model_NN_CeC.predict(X_train_CeC_mix)
Predictions_test_CeC_mix=model_NN_CeC.predict(X_test_CeC_mix)

precision_train_CeC_13_01 = r2_score(y_train_CeC_01,Predictions_train_CeC_01)
precision_test_CeC_13_01 = r2_score(y_test_CeC_01,Predictions_test_CeC_01)

precision_train_CeC_13_05 = r2_score(y_train_CeC_05,Predictions_train_CeC_05)
precision_test_CeC_13_05 = r2_score(y_test_CeC_05,Predictions_test_CeC_05)

precision_train_CeC_13_13 = r2_score(y_train_CeC_13,Predictions_train_CeC_13)
precision_test_CeC_13_13 = r2_score(y_test_CeC_13,Predictions_test_CeC_13)

precision_train_CeC_13_20 = r2_score(y_train_CeC_20,Predictions_train_CeC_20)
precision_test_CeC_13_20 = r2_score(y_test_CeC_20,Predictions_test_CeC_20)

precision_train_CeC_13_mix = r2_score(y_train_CeC_mix,Predictions_train_CeC_mix)
precision_test_CeC_13_mix = r2_score(y_test_CeC_mix,Predictions_test_CeC_mix)

# Dataset 20

model_NN_CeC.fit(X_train_CeC_20, y_train_CeC_20,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeC_01=model_NN_CeC.predict(X_train_CeC_01)
Predictions_test_CeC_01=model_NN_CeC.predict(X_test_CeC_01)

Predictions_train_CeC_05=model_NN_CeC.predict(X_train_CeC_05)
Predictions_test_CeC_05=model_NN_CeC.predict(X_test_CeC_05)

Predictions_train_CeC_13=model_NN_CeC.predict(X_train_CeC_13)
Predictions_test_CeC_13=model_NN_CeC.predict(X_test_CeC_13)

Predictions_train_CeC_20=model_NN_CeC.predict(X_train_CeC_20)
Predictions_test_CeC_20=model_NN_CeC.predict(X_test_CeC_20)

Predictions_train_CeC_mix=model_NN_CeC.predict(X_train_CeC_mix)
Predictions_test_CeC_mix=model_NN_CeC.predict(X_test_CeC_mix)

precision_train_CeC_20_01 = r2_score(y_train_CeC_01,Predictions_train_CeC_01)
precision_test_CeC_20_01 = r2_score(y_test_CeC_01,Predictions_test_CeC_01)

precision_train_CeC_20_05 = r2_score(y_train_CeC_05,Predictions_train_CeC_05)
precision_test_CeC_20_05 = r2_score(y_test_CeC_05,Predictions_test_CeC_05)

precision_train_CeC_20_13 = r2_score(y_train_CeC_13,Predictions_train_CeC_13)
precision_test_CeC_20_13 = r2_score(y_test_CeC_13,Predictions_test_CeC_13)

precision_train_CeC_20_20 = r2_score(y_train_CeC_20,Predictions_train_CeC_20)
precision_test_CeC_20_20 = r2_score(y_test_CeC_20,Predictions_test_CeC_20)

precision_train_CeC_20_mix = r2_score(y_train_CeC_mix,Predictions_train_CeC_mix)
precision_test_CeC_20_mix = r2_score(y_test_CeC_mix,Predictions_test_CeC_mix)

# Dataset mix

model_NN_CeC.fit(X_train_CeC_mix, y_train_CeC_mix,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CeC_01=model_NN_CeC.predict(X_train_CeC_01)
Predictions_test_CeC_01=model_NN_CeC.predict(X_test_CeC_01)

Predictions_train_CeC_05=model_NN_CeC.predict(X_train_CeC_05)
Predictions_test_CeC_05=model_NN_CeC.predict(X_test_CeC_05)

Predictions_train_CeC_13=model_NN_CeC.predict(X_train_CeC_13)
Predictions_test_CeC_13=model_NN_CeC.predict(X_test_CeC_13)

Predictions_train_CeC_20=model_NN_CeC.predict(X_train_CeC_20)
Predictions_test_CeC_20=model_NN_CeC.predict(X_test_CeC_20)

Predictions_train_CeC_mix=model_NN_CeC.predict(X_train_CeC_mix)
Predictions_test_CeC_mix=model_NN_CeC.predict(X_test_CeC_mix)

precision_train_CeC_mix_01 = r2_score(y_train_CeC_01,Predictions_train_CeC_01)
precision_test_CeC_mix_01 = r2_score(y_test_CeC_01,Predictions_test_CeC_01)

precision_train_CeC_mix_05 = r2_score(y_train_CeC_05,Predictions_train_CeC_05)
precision_test_CeC_mix_05 = r2_score(y_test_CeC_05,Predictions_test_CeC_05)

precision_train_CeC_mix_13 = r2_score(y_train_CeC_13,Predictions_train_CeC_13)
precision_test_CeC_mix_13 = r2_score(y_test_CeC_13,Predictions_test_CeC_13)

precision_train_CeC_mix_20 = r2_score(y_train_CeC_20,Predictions_train_CeC_20)
precision_test_CeC_mix_20 = r2_score(y_test_CeC_20,Predictions_test_CeC_20)

precision_train_CeC_mix_mix = r2_score(y_train_CeC_mix,Predictions_train_CeC_mix)
precision_test_CeC_mix_mix = r2_score(y_test_CeC_mix,Predictions_test_CeC_mix)

826/826 [==============================] - 0s 298us/step


,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix
Score train_01,0.470440,-28.400740,-155.480651,-132.837843,-34.332812
Score test_01,0.470050,-28.672149,-156.632536,-133.671185,-34.603626
Score train_05,-35.820820,0.907145,-33.443765,-17.252646,-9.614009
Score test_05,-34.257162,0.913770,-32.138680,-16.652489,-9.257824
Score train_13,-1111.127752,-17.354271,0.610853,-4.472878,-4.849708
Score test_13,-1117.135800,-17.484220,0.604257,-4.545122,-4.865984
Score train_20,-49.541994,-66.942915,-5.390509,0.690793,-6.103439
Score test_20,-49.367038,-66.994992,-5.400373,0.689132,-6.107861
Score train_mix,-2.289010,0.207955,0.456450,0.565560,0.788414
Score test_mix,-2.337927,0.200618,0.455296,0.570830,0.788042


In [52]:
input_shape = X_CoR_01.shape[1]
output_shape = 1

# we set up our input layer
inputs = Input(shape=(input_shape,))

# we add 3 hidden layers with diminishing size. This is a common practice in designing a neural network
# as the features get more and more abstracted, we need less and less neurons.

h = Dense(16, activation="relu")(inputs)
h = Dense(8, activation="relu")(h)
h = Dense(4, activation="relu")(h)

# and finally we use the ReLU activation function on the output layer
out = Dense(output_shape, activation="relu")(h)

model_NN_CoR = Sequential()
model_NN_CoR.add(Dense(units=5, input_dim=14, kernel_initializer='normal', activation='relu'))
model_NN_CoR.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
model_NN_CoR.add(Dense(1, kernel_initializer='normal'))
model_NN_CoR.compile(loss='mean_squared_error', optimizer='adam')

# Dataset 01

model_NN_CoR.fit(X_train_CoR_01, y_train_CoR_01,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoR_01=model_NN_CoR.predict(X_train_CoR_01)
Predictions_test_CoR_01=model_NN_CoR.predict(X_test_CoR_01)

Predictions_train_CoR_05=model_NN_CoR.predict(X_train_CoR_05)
Predictions_test_CoR_05=model_NN_CoR.predict(X_test_CoR_05)

Predictions_train_CoR_13=model_NN_CoR.predict(X_train_CoR_13)
Predictions_test_CoR_13=model_NN_CoR.predict(X_test_CoR_13)

Predictions_train_CoR_20=model_NN_CoR.predict(X_train_CoR_20)
Predictions_test_CoR_20=model_NN_CoR.predict(X_test_CoR_20)

Predictions_train_CoR_mix=model_NN_CoR.predict(X_train_CoR_mix)
Predictions_test_CoR_mix=model_NN_CoR.predict(X_test_CoR_mix)


precision_train_CoR_01_01 = r2_score(y_train_CoR_01,Predictions_train_CoR_01)
precision_test_CoR_01_01 = r2_score(y_test_CoR_01,Predictions_test_CoR_01)

precision_train_CoR_01_05 = r2_score(y_train_CoR_05,Predictions_train_CoR_05)
precision_test_CoR_01_05 = r2_score(y_test_CoR_05,Predictions_test_CoR_05)

precision_train_CoR_01_13 = r2_score(y_train_CoR_13,Predictions_train_CoR_13)
precision_test_CoR_01_13 = r2_score(y_test_CoR_13,Predictions_test_CoR_13)

precision_train_CoR_01_20 = r2_score(y_train_CoR_20,Predictions_train_CoR_20)
precision_test_CoR_01_20 = r2_score(y_test_CoR_20,Predictions_test_CoR_20)

precision_train_CoR_01_mix = r2_score(y_train_CoR_mix,Predictions_train_CoR_mix)
precision_test_CoR_01_mix = r2_score(y_test_CoR_mix,Predictions_test_CoR_mix)

# Dataset 05

model_NN_CoR.fit(X_train_CoR_05, y_train_CoR_05,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoR_01=model_NN_CoR.predict(X_train_CoR_01)
Predictions_test_CoR_01=model_NN_CoR.predict(X_test_CoR_01)

Predictions_train_CoR_05=model_NN_CoR.predict(X_train_CoR_05)
Predictions_test_CoR_05=model_NN_CoR.predict(X_test_CoR_05)

Predictions_train_CoR_13=model_NN_CoR.predict(X_train_CoR_13)
Predictions_test_CoR_13=model_NN_CoR.predict(X_test_CoR_13)

Predictions_train_CoR_20=model_NN_CoR.predict(X_train_CoR_20)
Predictions_test_CoR_20=model_NN_CoR.predict(X_test_CoR_20)

Predictions_train_CoR_mix=model_NN_CoR.predict(X_train_CoR_mix)
Predictions_test_CoR_mix=model_NN_CoR.predict(X_test_CoR_mix)

precision_train_CoR_05_01 = r2_score(y_train_CoR_01,Predictions_train_CoR_01)
precision_test_CoR_05_01 = r2_score(y_test_CoR_01,Predictions_test_CoR_01)

precision_train_CoR_05_05 = r2_score(y_train_CoR_05,Predictions_train_CoR_05)
precision_test_CoR_05_05 = r2_score(y_test_CoR_05,Predictions_test_CoR_05)

precision_train_CoR_05_13 = r2_score(y_train_CoR_13,Predictions_train_CoR_13)
precision_test_CoR_05_13 = r2_score(y_test_CoR_13,Predictions_test_CoR_13)

precision_train_CoR_05_20 = r2_score(y_train_CoR_20,Predictions_train_CoR_20)
precision_test_CoR_05_20 = r2_score(y_test_CoR_20,Predictions_test_CoR_20)

precision_train_CoR_05_mix = r2_score(y_train_CoR_mix,Predictions_train_CoR_mix)
precision_test_CoR_05_mix = r2_score(y_test_CoR_mix,Predictions_test_CoR_mix)

# Dataset 13

model_NN_CoR.fit(X_train_CoR_13, y_train_CoR_13,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoR_01=model_NN_CoR.predict(X_train_CoR_01)
Predictions_test_CoR_01=model_NN_CoR.predict(X_test_CoR_01)

Predictions_train_CoR_05=model_NN_CoR.predict(X_train_CoR_05)
Predictions_test_CoR_05=model_NN_CoR.predict(X_test_CoR_05)

Predictions_train_CoR_13=model_NN_CoR.predict(X_train_CoR_13)
Predictions_test_CoR_13=model_NN_CoR.predict(X_test_CoR_13)

Predictions_train_CoR_20=model_NN_CoR.predict(X_train_CoR_20)
Predictions_test_CoR_20=model_NN_CoR.predict(X_test_CoR_20)

Predictions_train_CoR_mix=model_NN_CoR.predict(X_train_CoR_mix)
Predictions_test_CoR_mix=model_NN_CoR.predict(X_test_CoR_mix)

precision_train_CoR_13_01 = r2_score(y_train_CoR_01,Predictions_train_CoR_01)
precision_test_CoR_13_01 = r2_score(y_test_CoR_01,Predictions_test_CoR_01)

precision_train_CoR_13_05 = r2_score(y_train_CoR_05,Predictions_train_CoR_05)
precision_test_CoR_13_05 = r2_score(y_test_CoR_05,Predictions_test_CoR_05)

precision_train_CoR_13_13 = r2_score(y_train_CoR_13,Predictions_train_CoR_13)
precision_test_CoR_13_13 = r2_score(y_test_CoR_13,Predictions_test_CoR_13)

precision_train_CoR_13_20 = r2_score(y_train_CoR_20,Predictions_train_CoR_20)
precision_test_CoR_13_20 = r2_score(y_test_CoR_20,Predictions_test_CoR_20)

precision_train_CoR_13_mix = r2_score(y_train_CoR_mix,Predictions_train_CoR_mix)
precision_test_CoR_13_mix = r2_score(y_test_CoR_mix,Predictions_test_CoR_mix)

# Dataset 20

model_NN_CoR.fit(X_train_CoR_20, y_train_CoR_20,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoR_01=model_NN_CoR.predict(X_train_CoR_01)
Predictions_test_CoR_01=model_NN_CoR.predict(X_test_CoR_01)

Predictions_train_CoR_05=model_NN_CoR.predict(X_train_CoR_05)
Predictions_test_CoR_05=model_NN_CoR.predict(X_test_CoR_05)

Predictions_train_CoR_13=model_NN_CoR.predict(X_train_CoR_13)
Predictions_test_CoR_13=model_NN_CoR.predict(X_test_CoR_13)

Predictions_train_CoR_20=model_NN_CoR.predict(X_train_CoR_20)
Predictions_test_CoR_20=model_NN_CoR.predict(X_test_CoR_20)

Predictions_train_CoR_mix=model_NN_CoR.predict(X_train_CoR_mix)
Predictions_test_CoR_mix=model_NN_CoR.predict(X_test_CoR_mix)

precision_train_CoR_20_01 = r2_score(y_train_CoR_01,Predictions_train_CoR_01)
precision_test_CoR_20_01 = r2_score(y_test_CoR_01,Predictions_test_CoR_01)

precision_train_CoR_20_05 = r2_score(y_train_CoR_05,Predictions_train_CoR_05)
precision_test_CoR_20_05 = r2_score(y_test_CoR_05,Predictions_test_CoR_05)

precision_train_CoR_20_13 = r2_score(y_train_CoR_13,Predictions_train_CoR_13)
precision_test_CoR_20_13 = r2_score(y_test_CoR_13,Predictions_test_CoR_13)

precision_train_CoR_20_20 = r2_score(y_train_CoR_20,Predictions_train_CoR_20)
precision_test_CoR_20_20 = r2_score(y_test_CoR_20,Predictions_test_CoR_20)

precision_train_CoR_20_mix = r2_score(y_train_CoR_mix,Predictions_train_CoR_mix)
precision_test_CoR_20_mix = r2_score(y_test_CoR_mix,Predictions_test_CoR_mix)

# Dataset mix

model_NN_CoR.fit(X_train_CoR_mix, y_train_CoR_mix,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoR_01=model_NN_CoR.predict(X_train_CoR_01)
Predictions_test_CoR_01=model_NN_CoR.predict(X_test_CoR_01)

Predictions_train_CoR_05=model_NN_CoR.predict(X_train_CoR_05)
Predictions_test_CoR_05=model_NN_CoR.predict(X_test_CoR_05)

Predictions_train_CoR_13=model_NN_CoR.predict(X_train_CoR_13)
Predictions_test_CoR_13=model_NN_CoR.predict(X_test_CoR_13)

Predictions_train_CoR_20=model_NN_CoR.predict(X_train_CoR_20)
Predictions_test_CoR_20=model_NN_CoR.predict(X_test_CoR_20)

Predictions_train_CoR_mix=model_NN_CoR.predict(X_train_CoR_mix)
Predictions_test_CoR_mix=model_NN_CoR.predict(X_test_CoR_mix)

precision_train_CoR_mix_01 = r2_score(y_train_CoR_01,Predictions_train_CoR_01)
precision_test_CoR_mix_01 = r2_score(y_test_CoR_01,Predictions_test_CoR_01)

precision_train_CoR_mix_05 = r2_score(y_train_CoR_05,Predictions_train_CoR_05)
precision_test_CoR_mix_05 = r2_score(y_test_CoR_05,Predictions_test_CoR_05)

precision_train_CoR_mix_13 = r2_score(y_train_CoR_13,Predictions_train_CoR_13)
precision_test_CoR_mix_13 = r2_score(y_test_CoR_13,Predictions_test_CoR_13)

precision_train_CoR_mix_20 = r2_score(y_train_CoR_20,Predictions_train_CoR_20)
precision_test_CoR_mix_20 = r2_score(y_test_CoR_20,Predictions_test_CoR_20)

precision_train_CoR_mix_mix = r2_score(y_train_CoR_mix,Predictions_train_CoR_mix)
precision_test_CoR_mix_mix = r2_score(y_test_CoR_mix,Predictions_test_CoR_mix)

1035/1035 [==============================] - 0s 284us/step


,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix
Score train_01,0.884525,-165.476530,-1.321982,-373.736265,-2.276347
Score test_01,0.882079,-168.943299,-1.368352,-381.569546,-2.327475
Score train_05,-176.756338,0.868196,-227.972509,-47.177736,-1.577401
Score test_05,-176.819582,0.868945,-228.072536,-47.232937,-1.585752
Score train_13,-2.930696,-244.451136,0.810487,-521.134109,-6.377700
Score test_13,-2.934934,-244.897338,0.811127,-522.039827,-6.410070
Score train_20,-453.732761,-50.942079,-521.768474,0.861154,-2.084324
Score test_20,-450.477246,-50.589011,-518.073198,0.861066,-2.049410
Score train_mix,-1.063631,0.035761,-1.465243,-1.107200,0.937962
Score test_mix,-1.065000,0.032097,-1.469069,-1.119937,0.937301


In [53]:
input_shape = X_CoC_01.shape[1]
output_shape = 1

# we set up our input layer
inputs = Input(shape=(input_shape,))

# we add 3 hidden layers with diminishing size. This is a common practice in designing a neural network
# as the features get more and more abstracted, we need less and less neurons.

h = Dense(16, activation="relu")(inputs)
h = Dense(8, activation="relu")(h)
h = Dense(4, activation="relu")(h)

# and finally we use the ReLU activation function on the output layer
out = Dense(output_shape, activation="relu")(h)

model_NN_CoC = Sequential()
model_NN_CoC.add(Dense(units=5, input_dim=14, kernel_initializer='normal', activation='relu'))
model_NN_CoC.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
model_NN_CoC.add(Dense(1, kernel_initializer='normal'))
model_NN_CoC.compile(loss='mean_squared_error', optimizer='adam')

# Dataset 01

model_NN_CoC.fit(X_train_CoC_01, y_train_CoC_01,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoC_01=model_NN_CoC.predict(X_train_CoC_01)
Predictions_test_CoC_01=model_NN_CoC.predict(X_test_CoC_01)

Predictions_train_CoC_05=model_NN_CoC.predict(X_train_CoC_05)
Predictions_test_CoC_05=model_NN_CoC.predict(X_test_CoC_05)

Predictions_train_CoC_13=model_NN_CoC.predict(X_train_CoC_13)
Predictions_test_CoC_13=model_NN_CoC.predict(X_test_CoC_13)

Predictions_train_CoC_20=model_NN_CoC.predict(X_train_CoC_20)
Predictions_test_CoC_20=model_NN_CoC.predict(X_test_CoC_20)

Predictions_train_CoC_mix=model_NN_CoC.predict(X_train_CoC_mix)
Predictions_test_CoC_mix=model_NN_CoC.predict(X_test_CoC_mix)


precision_train_CoC_01_01 = r2_score(y_train_CoC_01,Predictions_train_CoC_01)
precision_test_CoC_01_01 = r2_score(y_test_CoC_01,Predictions_test_CoC_01)

precision_train_CoC_01_05 = r2_score(y_train_CoC_05,Predictions_train_CoC_05)
precision_test_CoC_01_05 = r2_score(y_test_CoC_05,Predictions_test_CoC_05)

precision_train_CoC_01_13 = r2_score(y_train_CoC_13,Predictions_train_CoC_13)
precision_test_CoC_01_13 = r2_score(y_test_CoC_13,Predictions_test_CoC_13)

precision_train_CoC_01_20 = r2_score(y_train_CoC_20,Predictions_train_CoC_20)
precision_test_CoC_01_20 = r2_score(y_test_CoC_20,Predictions_test_CoC_20)

precision_train_CoC_01_mix = r2_score(y_train_CoC_mix,Predictions_train_CoC_mix)
precision_test_CoC_01_mix = r2_score(y_test_CoC_mix,Predictions_test_CoC_mix)

# Dataset 05

model_NN_CoC.fit(X_train_CoC_05, y_train_CoC_05,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoC_01=model_NN_CoC.predict(X_train_CoC_01)
Predictions_test_CoC_01=model_NN_CoC.predict(X_test_CoC_01)

Predictions_train_CoC_05=model_NN_CoC.predict(X_train_CoC_05)
Predictions_test_CoC_05=model_NN_CoC.predict(X_test_CoC_05)

Predictions_train_CoC_13=model_NN_CoC.predict(X_train_CoC_13)
Predictions_test_CoC_13=model_NN_CoC.predict(X_test_CoC_13)

Predictions_train_CoC_20=model_NN_CoC.predict(X_train_CoC_20)
Predictions_test_CoC_20=model_NN_CoC.predict(X_test_CoC_20)

Predictions_train_CoC_mix=model_NN_CoC.predict(X_train_CoC_mix)
Predictions_test_CoC_mix=model_NN_CoC.predict(X_test_CoC_mix)

precision_train_CoC_05_01 = r2_score(y_train_CoC_01,Predictions_train_CoC_01)
precision_test_CoC_05_01 = r2_score(y_test_CoC_01,Predictions_test_CoC_01)

precision_train_CoC_05_05 = r2_score(y_train_CoC_05,Predictions_train_CoC_05)
precision_test_CoC_05_05 = r2_score(y_test_CoC_05,Predictions_test_CoC_05)

precision_train_CoC_05_13 = r2_score(y_train_CoC_13,Predictions_train_CoC_13)
precision_test_CoC_05_13 = r2_score(y_test_CoC_13,Predictions_test_CoC_13)

precision_train_CoC_05_20 = r2_score(y_train_CoC_20,Predictions_train_CoC_20)
precision_test_CoC_05_20 = r2_score(y_test_CoC_20,Predictions_test_CoC_20)

precision_train_CoC_05_mix = r2_score(y_train_CoC_mix,Predictions_train_CoC_mix)
precision_test_CoC_05_mix = r2_score(y_test_CoC_mix,Predictions_test_CoC_mix)

# Dataset 13

model_NN_CoC.fit(X_train_CoC_13, y_train_CoC_13,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoC_01=model_NN_CoC.predict(X_train_CoC_01)
Predictions_test_CoC_01=model_NN_CoC.predict(X_test_CoC_01)

Predictions_train_CoC_05=model_NN_CoC.predict(X_train_CoC_05)
Predictions_test_CoC_05=model_NN_CoC.predict(X_test_CoC_05)

Predictions_train_CoC_13=model_NN_CoC.predict(X_train_CoC_13)
Predictions_test_CoC_13=model_NN_CoC.predict(X_test_CoC_13)

Predictions_train_CoC_20=model_NN_CoC.predict(X_train_CoC_20)
Predictions_test_CoC_20=model_NN_CoC.predict(X_test_CoC_20)

Predictions_train_CoC_mix=model_NN_CoC.predict(X_train_CoC_mix)
Predictions_test_CoC_mix=model_NN_CoC.predict(X_test_CoC_mix)

precision_train_CoC_13_01 = r2_score(y_train_CoC_01,Predictions_train_CoC_01)
precision_test_CoC_13_01 = r2_score(y_test_CoC_01,Predictions_test_CoC_01)

precision_train_CoC_13_05 = r2_score(y_train_CoC_05,Predictions_train_CoC_05)
precision_test_CoC_13_05 = r2_score(y_test_CoC_05,Predictions_test_CoC_05)

precision_train_CoC_13_13 = r2_score(y_train_CoC_13,Predictions_train_CoC_13)
precision_test_CoC_13_13 = r2_score(y_test_CoC_13,Predictions_test_CoC_13)

precision_train_CoC_13_20 = r2_score(y_train_CoC_20,Predictions_train_CoC_20)
precision_test_CoC_13_20 = r2_score(y_test_CoC_20,Predictions_test_CoC_20)

precision_train_CoC_13_mix = r2_score(y_train_CoC_mix,Predictions_train_CoC_mix)
precision_test_CoC_13_mix = r2_score(y_test_CoC_mix,Predictions_test_CoC_mix)

# Dataset 20

model_NN_CoC.fit(X_train_CoC_20, y_train_CoC_20,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoC_01=model_NN_CoC.predict(X_train_CoC_01)
Predictions_test_CoC_01=model_NN_CoC.predict(X_test_CoC_01)

Predictions_train_CoC_05=model_NN_CoC.predict(X_train_CoC_05)
Predictions_test_CoC_05=model_NN_CoC.predict(X_test_CoC_05)

Predictions_train_CoC_13=model_NN_CoC.predict(X_train_CoC_13)
Predictions_test_CoC_13=model_NN_CoC.predict(X_test_CoC_13)

Predictions_train_CoC_20=model_NN_CoC.predict(X_train_CoC_20)
Predictions_test_CoC_20=model_NN_CoC.predict(X_test_CoC_20)

Predictions_train_CoC_mix=model_NN_CoC.predict(X_train_CoC_mix)
Predictions_test_CoC_mix=model_NN_CoC.predict(X_test_CoC_mix)

precision_train_CoC_20_01 = r2_score(y_train_CoC_01,Predictions_train_CoC_01)
precision_test_CoC_20_01 = r2_score(y_test_CoC_01,Predictions_test_CoC_01)

precision_train_CoC_20_05 = r2_score(y_train_CoC_05,Predictions_train_CoC_05)
precision_test_CoC_20_05 = r2_score(y_test_CoC_05,Predictions_test_CoC_05)

precision_train_CoC_20_13 = r2_score(y_train_CoC_13,Predictions_train_CoC_13)
precision_test_CoC_20_13 = r2_score(y_test_CoC_13,Predictions_test_CoC_13)

precision_train_CoC_20_20 = r2_score(y_train_CoC_20,Predictions_train_CoC_20)
precision_test_CoC_20_20 = r2_score(y_test_CoC_20,Predictions_test_CoC_20)

precision_train_CoC_20_mix = r2_score(y_train_CoC_mix,Predictions_train_CoC_mix)
precision_test_CoC_20_mix = r2_score(y_test_CoC_mix,Predictions_test_CoC_mix)

# Dataset mix

model_NN_CoC.fit(X_train_CoC_mix, y_train_CoC_mix,batch_size = 20, epochs = 100, verbose=0)

Predictions_train_CoC_01=model_NN_CoC.predict(X_train_CoC_01)
Predictions_test_CoC_01=model_NN_CoC.predict(X_test_CoC_01)

Predictions_train_CoC_05=model_NN_CoC.predict(X_train_CoC_05)
Predictions_test_CoC_05=model_NN_CoC.predict(X_test_CoC_05)

Predictions_train_CoC_13=model_NN_CoC.predict(X_train_CoC_13)
Predictions_test_CoC_13=model_NN_CoC.predict(X_test_CoC_13)

Predictions_train_CoC_20=model_NN_CoC.predict(X_train_CoC_20)
Predictions_test_CoC_20=model_NN_CoC.predict(X_test_CoC_20)

Predictions_train_CoC_mix=model_NN_CoC.predict(X_train_CoC_mix)
Predictions_test_CoC_mix=model_NN_CoC.predict(X_test_CoC_mix)

precision_train_CoC_mix_01 = r2_score(y_train_CoC_01,Predictions_train_CoC_01)
precision_test_CoC_mix_01 = r2_score(y_test_CoC_01,Predictions_test_CoC_01)

precision_train_CoC_mix_05 = r2_score(y_train_CoC_05,Predictions_train_CoC_05)
precision_test_CoC_mix_05 = r2_score(y_test_CoC_05,Predictions_test_CoC_05)

precision_train_CoC_mix_13 = r2_score(y_train_CoC_13,Predictions_train_CoC_13)
precision_test_CoC_mix_13 = r2_score(y_test_CoC_13,Predictions_test_CoC_13)

precision_train_CoC_mix_20 = r2_score(y_train_CoC_20,Predictions_train_CoC_20)
precision_test_CoC_mix_20 = r2_score(y_test_CoC_20,Predictions_test_CoC_20)

precision_train_CoC_mix_mix = r2_score(y_train_CoC_mix,Predictions_train_CoC_mix)
precision_test_CoC_mix_mix = r2_score(y_test_CoC_mix,Predictions_test_CoC_mix)

1034/1034 [==============================] - 0s 227us/step


,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.904125,-37.327789,-25.537707,-169.471265,-49.155096
Score test_01,0.904730,-37.062153,-25.251426,-167.517375,-48.628569
Score train_05,-168.020663,0.954419,-45.440504,-566.905258,-8.254128
Score test_05,-162.456089,0.954155,-43.909585,-548.260714,-7.994801
Score train_13,-2.538402,-65.434938,0.971484,-356.488998,-1.535249
Score test_13,-2.529424,-65.353953,0.971242,-355.866833,-1.531164
Score train_20,-683.692538,-1879.528354,-145.582539,0.157898,-14.446996
Score test_20,-739.977499,-2035.686274,-157.459790,0.133625,-15.713081
Score train_mix,0.440100,0.478472,0.701341,-1.704421,0.911824
Score test_mix,0.441125,0.475937,0.699999,-1.707212,0.910761


In [54]:
# Bilan 

# Tableau de bilan pour comparer les scores.

data = {
        'Center_random_01': [precision_train_CeR_01_01, precision_test_CeR_01_01, precision_train_CeR_01_05, precision_test_CeR_01_05, precision_train_CeR_01_13, precision_test_CeR_01_13, precision_train_CeR_01_20, precision_test_CeR_01_20, precision_train_CeR_01_mix, precision_test_CeR_01_mix], 
        'Center_random_05': [precision_train_CeR_05_01, precision_test_CeR_05_01, precision_train_CeR_05_05, precision_test_CeR_05_05, precision_train_CeR_05_13, precision_test_CeR_05_13, precision_train_CeR_05_20, precision_test_CeR_05_20, precision_train_CeR_05_mix, precision_test_CeR_05_mix],
        'Center_random_13': [precision_train_CeR_13_01, precision_test_CeR_13_01, precision_train_CeR_13_05, precision_test_CeR_13_05, precision_train_CeR_13_13, precision_test_CeR_13_13, precision_train_CeR_13_20, precision_test_CeR_13_20, precision_train_CeR_13_mix, precision_test_CeR_13_mix],
        'Center_random_20': [precision_train_CeR_20_01, precision_test_CeR_20_01, precision_train_CeR_20_05, precision_test_CeR_20_05, precision_train_CeR_20_13, precision_test_CeR_20_13, precision_train_CeR_20_20, precision_test_CeR_20_20, precision_train_CeR_20_mix, precision_test_CeR_20_mix],
        'Center_random_mix': [precision_train_CeR_mix_01, precision_test_CeR_mix_01, precision_train_CeR_mix_05, precision_test_CeR_mix_05, precision_train_CeR_mix_13, precision_test_CeR_mix_13, precision_train_CeR_mix_20, precision_test_CeR_mix_20, precision_train_CeR_mix_mix, precision_test_CeR_mix_mix],
    
        'Center_cluster_01': [precision_train_CeC_01_01, precision_test_CeC_01_01, precision_train_CeC_01_05, precision_test_CeC_01_05, precision_train_CeC_01_13, precision_test_CeC_01_13, precision_train_CeC_01_20, precision_test_CeC_01_20, precision_train_CeC_01_mix, precision_test_CeC_01_mix], 
        'Center_cluster_05': [precision_train_CeC_05_01, precision_test_CeC_05_01, precision_train_CeC_05_05, precision_test_CeC_05_05, precision_train_CeC_05_13, precision_test_CeC_05_13, precision_train_CeC_05_20, precision_test_CeC_05_20, precision_train_CeC_05_mix, precision_test_CeC_05_mix],
        'Center_cluster_13': [precision_train_CeC_13_01, precision_test_CeC_13_01, precision_train_CeC_13_05, precision_test_CeC_13_05, precision_train_CeC_13_13, precision_test_CeC_13_13, precision_train_CeC_13_20, precision_test_CeC_13_20, precision_train_CeC_13_mix, precision_test_CeC_13_mix],
        'Center_cluster_20': [precision_train_CeC_20_01, precision_test_CeC_20_01, precision_train_CeC_20_05, precision_test_CeC_20_05, precision_train_CeC_20_13, precision_test_CeC_20_13, precision_train_CeC_20_20, precision_test_CeC_20_20, precision_train_CeC_20_mix, precision_test_CeC_20_mix],
        'Center_cluster_mix': [precision_train_CeC_mix_01, precision_test_CeC_mix_01, precision_train_CeC_mix_05, precision_test_CeC_mix_05, precision_train_CeC_mix_13, precision_test_CeC_mix_13, precision_train_CeC_mix_20, precision_test_CeC_mix_20, precision_train_CeC_mix_mix, precision_test_CeC_mix_mix],
        
        'Coin_random_01': [precision_train_CoR_01_01, precision_test_CoR_01_01, precision_train_CoR_01_05, precision_test_CoR_01_05, precision_train_CoR_01_13, precision_test_CoR_01_13, precision_train_CoR_01_20, precision_test_CoR_01_20, precision_train_CoR_01_mix, precision_test_CoR_01_mix], 
        'Coin_random_05': [precision_train_CoR_05_01, precision_test_CoR_05_01, precision_train_CoR_05_05, precision_test_CoR_05_05, precision_train_CoR_05_13, precision_test_CoR_05_13, precision_train_CoR_05_20, precision_test_CoR_05_20, precision_train_CoR_05_mix, precision_test_CoR_05_mix],
        'Coin_random_13': [precision_train_CoR_13_01, precision_test_CoR_13_01, precision_train_CoR_13_05, precision_test_CoR_13_05, precision_train_CoR_13_13, precision_test_CoR_13_13, precision_train_CoR_13_20, precision_test_CoR_13_20, precision_train_CoR_13_mix, precision_test_CoR_13_mix],
        'Coin_random_20': [precision_train_CoR_20_01, precision_test_CoR_20_01, precision_train_CoR_20_05, precision_test_CoR_20_05, precision_train_CoR_20_13, precision_test_CoR_20_13, precision_train_CoR_20_20, precision_test_CoR_20_20, precision_train_CoR_20_mix, precision_test_CoR_20_mix],
        'Coin_random_mix': [precision_train_CoR_mix_01, precision_test_CoR_mix_01, precision_train_CoR_mix_05, precision_test_CoR_mix_05, precision_train_CoR_mix_13, precision_test_CoR_mix_13, precision_train_CoR_mix_20, precision_test_CoR_mix_20, precision_train_CoR_mix_mix, precision_test_CoR_mix_mix],
    
        'Coin_cluster_01': [precision_train_CoC_01_01, precision_test_CoC_01_01, precision_train_CoC_01_05, precision_test_CoC_01_05, precision_train_CoC_01_13, precision_test_CoC_01_13, precision_train_CoC_01_20, precision_test_CoC_01_20, precision_train_CoC_01_mix, precision_test_CoC_01_mix], 
        'Coin_cluster_05': [precision_train_CoC_05_01, precision_test_CoC_05_01, precision_train_CoC_05_05, precision_test_CoC_05_05, precision_train_CoC_05_13, precision_test_CoC_05_13, precision_train_CoC_05_20, precision_test_CoC_05_20, precision_train_CoC_05_mix, precision_test_CoC_05_mix],
        'Coin_cluster_13': [precision_train_CoC_13_01, precision_test_CoC_13_01, precision_train_CoC_13_05, precision_test_CoC_13_05, precision_train_CoC_13_13, precision_test_CoC_13_13, precision_train_CoC_13_20, precision_test_CoC_13_20, precision_train_CoC_13_mix, precision_test_CoC_13_mix],
        'Coin_cluster_20': [precision_train_CoC_20_01, precision_test_CoC_20_01, precision_train_CoC_20_05, precision_test_CoC_20_05, precision_train_CoC_20_13, precision_test_CoC_20_13, precision_train_CoC_20_20, precision_test_CoC_20_20, precision_train_CoC_20_mix, precision_test_CoC_20_mix],
        'Coin_cluster_mix': [precision_train_CoC_mix_01, precision_test_CoC_mix_01, precision_train_CoC_mix_05, precision_test_CoC_mix_05, precision_train_CoC_mix_13, precision_test_CoC_mix_13, precision_train_CoC_mix_20, precision_test_CoC_mix_20, precision_train_CoC_mix_mix, precision_test_CoC_mix_mix]
       }

pd.DataFrame( data, 
             index = ['Score train_01','Score test_01', 'Score train_05','Score test_05', 'Score train_13','Score test_13','Score train_20','Score test_20', 'Score train_mix','Score test_mix'])

,Center_random_01,Center_random_05,Center_random_13,Center_random_20,Center_random_mix,Center_cluster_01,Center_cluster_05,Center_cluster_13,Center_cluster_20,Center_cluster_mix,Coin_random_01,Coin_random_05,Coin_random_13,Coin_random_20,Coin_random_mix,Coin_cluster_01,Coin_cluster_05,Coin_cluster_13,Coin_cluster_20,Coin_cluster_mix
Score train_01,0.630612,-192.691212,0.300375,-65.706674,-3.035023,0.470440,-28.400740,-155.480651,-132.837843,-34.332812,0.884525,-165.476530,-1.321982,-373.736265,-2.276347,0.904125,-37.327789,-25.537707,-169.471265,-49.155096
Score test_01,0.608815,-204.676989,0.254352,-69.820991,-3.252302,0.470050,-28.672149,-156.632536,-133.671185,-34.603626,0.882079,-168.943299,-1.368352,-381.569546,-2.327475,0.904730,-37.062153,-25.251426,-167.517375,-48.628569
Score train_05,-144.563585,0.650190,-92.218183,-19.307212,-2.615155,-35.820820,0.907145,-33.443765,-17.252646,-9.614009,-176.756338,0.868196,-227.972509,-47.177736,-1.577401,-168.020663,0.954419,-45.440504,-566.905258,-8.254128
Score test_05,-143.343578,0.643997,-91.378291,-19.130758,-2.580816,-34.257162,0.913770,-32.138680,-16.652489,-9.257824,-176.819582,0.868945,-228.072536,-47.232937,-1.585752,-162.456089,0.954155,-43.909585,-548.260714,-7.994801
Score train_13,0.205964,-200.640783,0.731093,-63.612914,-2.352775,-1111.127752,-17.354271,0.610853,-4.472878,-4.849708,-2.930696,-244.451136,0.810487,-521.134109,-6.377700,-2.538402,-65.434938,0.971484,-356.488998,-1.535249
Score test_13,0.192428,-203.192507,0.723526,-64.437256,-2.335258,-1117.135800,-17.484220,0.604257,-4.545122,-4.865984,-2.934934,-244.897338,0.811127,-522.039827,-6.410070,-2.529424,-65.353953,0.971242,-355.866833,-1.531164
Score train_20,-15.601843,-18.057168,-12.547540,0.857383,0.175595,-49.541994,-66.942915,-5.390509,0.690793,-6.103439,-453.732761,-50.942079,-521.768474,0.861154,-2.084324,-683.692538,-1879.528354,-145.582539,0.157898,-14.446996
Score test_20,-15.411784,-17.941156,-12.380212,0.855195,0.204611,-49.367038,-66.994992,-5.400373,0.689132,-6.107861,-450.477246,-50.589011,-518.073198,0.861066,-2.049410,-739.977499,-2035.686274,-157.459790,0.133625,-15.713081
Score train_mix,-0.979499,-2.161126,-0.408077,-0.080718,0.794808,-2.289010,0.207955,0.456450,0.565560,0.788414,-1.063631,0.035761,-1.465243,-1.107200,0.937962,0.440100,0.478472,0.701341,-1.704421,0.911824
Score test_mix,-1.010008,-2.107516,-0.431533,-0.070094,0.793812,-2.337927,0.200618,0.455296,0.570830,0.788042,-1.065000,0.032097,-1.469069,-1.119937,0.937301,0.441125,0.475937,0.699999,-1.707212,0.910761
